In [1]:
# SNIPPET S0: RAW SCAN INDEX BUILDER
# Creates unified scan index from OASIS-2 (P1+P2) and OASIS-3 raw directories

import os
import re
from pathlib import Path
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def get_extension(filename):
    """Extract file extension, handling .nii.gz as special case."""
    if filename.endswith('.nii.gz'):
        return '.nii.gz'
    else:
        return Path(filename).suffix


def extract_digits_after_prefix(text, prefix):
    """
    Extract first sequence of digits after prefix in text.
    E.g., extract_digits_after_prefix("mpr-1.nifti.hdr", "mpr-") -> "1"
    """
    pattern = re.escape(prefix) + r'(\d+)'
    match = re.search(pattern, text)
    return match.group(1) if match else None


def walk_files(root_dir):
    """
    Walk directory tree and yield file metadata.
    Returns list of dicts with: full_path, filename, parent_dir, ext
    """
    rows = []
    if not os.path.exists(root_dir):
        print(f"WARNING: Root directory does not exist: {root_dir}")
        return rows
    
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            full_path = os.path.join(dirpath, fname)
            parent_dir = os.path.basename(dirpath)
            ext = get_extension(fname)
            rows.append({
                "full_path": full_path,
                "filename": fname,
                "parent_dir": parent_dir,
                "ext": ext,
            })
    return rows


def parse_oasis2_entry(entry, root_tag):
    """
    Parse OASIS-2 file entry into structured metadata.
    
    Args:
        entry: dict with full_path, filename, parent_dir, ext
        root_tag: "OAS2_RAW_PART1" or "OAS2_RAW_PART2"
    
    Returns:
        dict with parsed fields or None if not a valid NIfTI
    """
    path = entry["full_path"]
    fname = entry["filename"]
    ext = entry["ext"]
    
    # Only accept NIfTI-like extensions
    if ext not in [".hdr", ".img", ".nii", ".nii.gz"]:
        return None
    
    # Only keep .hdr files (not .img pairs) or standalone .nii/.nii.gz
    if ext == ".img":
        return None
    
    # Parse directory structure
    parts = Path(path).parts
    
    # Find root_tag index to extract subject-session directory
    try:
        idx = parts.index(root_tag)
    except ValueError:
        # root_tag not in path - skip
        return None
    
    if idx + 1 >= len(parts):
        return None
    
    subj_sess_dir = parts[idx + 1]  # e.g., "OAS2_0001_MR1"
    
    # Parse subject_id and visit_code
    tokens = subj_sess_dir.split("_")
    if len(tokens) < 3:
        # Unexpected format
        return None
    
    # Expected: ["OAS2", "0001", "MR1"]
    subject_id = "_".join(tokens[:2])  # "OAS2_0001"
    visit_code = tokens[-1]            # "MR1"
    
    # Extract run_id from filename
    name_lower = fname.lower()
    run_id = 1
    
    if "mpr-" in name_lower:
        run_str = extract_digits_after_prefix(name_lower, "mpr-")
        if run_str:
            run_id = int(run_str)
    
    # Determine if T1-weighted
    is_t1w = ("mpr" in name_lower) or ("t1" in name_lower)
    modality = "T1w" if is_t1w else "UNKNOWN"
    
    dataset = "OASIS2"
    source = "OASIS2_P1" if root_tag == "OAS2_RAW_PART1" else "OASIS2_P2"
    
    scan_uid = f"{dataset}|{subject_id}|{visit_code}|{run_id}|{fname}"
    
    return {
        "nifti_path": path,
        "dataset": dataset,
        "source": source,
        "filename": fname,
        "parent_dir": entry["parent_dir"],
        "ext": ext,
        "subject_id": subject_id,
        "visit_code": visit_code,
        "run_id": run_id,
        "modality": modality,
        "is_t1w": is_t1w,
        "scan_uid": scan_uid,
    }


def parse_oasis3_entry(entry):
    """
    Parse OASIS-3 file entry into structured metadata.
    
    Args:
        entry: dict with full_path, filename, parent_dir, ext
    
    Returns:
        dict with parsed fields or None if not a valid NIfTI
    """
    path = entry["full_path"]
    fname = entry["filename"]
    ext = entry["ext"]
    
    # Only accept .nii or .nii.gz
    if ext not in [".nii", ".nii.gz"]:
        return None
    
    # Parse directory structure
    parts = Path(path).parts
    
    # Find "oaisis3" index
    try:
        idx = parts.index("oaisis3")
    except ValueError:
        return None
    
    if idx + 1 >= len(parts):
        return None
    
    subj_sess_dir = parts[idx + 1]  # e.g., "OAS30001_MR_d0129"
    
    # Parse subject_id and visit_code
    tokens = subj_sess_dir.split("_")
    if len(tokens) < 3:
        return None
    
    # Expected: ["OAS30001", "MR", "d0129"]
    subject_id = tokens[0]     # "OAS30001"
    visit_code = tokens[-1]    # "d0129"
    
    # Extract run_id from filename
    name_lower = fname.lower()
    run_id = 1
    
    if "run-" in name_lower:
        run_str = extract_digits_after_prefix(name_lower, "run-")
        if run_str:
            run_id = int(run_str)
    
    # Determine if T1-weighted
    is_t1w = ("t1w" in name_lower) or ("_t1." in name_lower)
    modality = "T1w" if is_t1w else "UNKNOWN"
    
    dataset = "OASIS3"
    source = "OASIS3"
    
    scan_uid = f"{dataset}|{subject_id}|{visit_code}|{run_id}|{fname}"
    
    return {
        "nifti_path": path,
        "dataset": dataset,
        "source": source,
        "filename": fname,
        "parent_dir": entry["parent_dir"],
        "ext": ext,
        "subject_id": subject_id,
        "visit_code": visit_code,
        "run_id": run_id,
        "modality": modality,
        "is_t1w": is_t1w,
        "scan_uid": scan_uid,
    }


# ============================================================================
# MAIN S0 FLOW
# ============================================================================

print("=" * 70)
print("SNIPPET S0: RAW SCAN INDEX BUILDER")
print("=" * 70)

all_rows = []

# ----------------------------------------------------------------------------
# OASIS-2 PART 1
# ----------------------------------------------------------------------------
print("\n[1/3] Scanning OASIS-2 PART 1...")
root_o2_p1 = "/kaggle/input/oaisis-dataset-3-p1/OAS2_RAW_PART1"
entries_o2_p1 = walk_files(root_o2_p1)
print(f"  Found {len(entries_o2_p1)} files in directory tree")

for e in entries_o2_p1:
    row = parse_oasis2_entry(e, root_tag="OAS2_RAW_PART1")
    if row is not None:
        all_rows.append(row)

print(f"  Parsed {len([r for r in all_rows if r['source'] == 'OASIS2_P1'])} valid scans")

# ----------------------------------------------------------------------------
# OASIS-2 PART 2
# ----------------------------------------------------------------------------
print("\n[2/3] Scanning OASIS-2 PART 2...")
root_o2_p2 = "/kaggle/input/oaisis-3-p2/OAS2_RAW_PART2"
entries_o2_p2 = walk_files(root_o2_p2)
print(f"  Found {len(entries_o2_p2)} files in directory tree")

count_before = len(all_rows)
for e in entries_o2_p2:
    row = parse_oasis2_entry(e, root_tag="OAS2_RAW_PART2")
    if row is not None:
        all_rows.append(row)

print(f"  Parsed {len(all_rows) - count_before} valid scans")

# ----------------------------------------------------------------------------
# OASIS-3
# ----------------------------------------------------------------------------
print("\n[3/3] Scanning OASIS-3...")
root_o3 = "/kaggle/input/oaisis-3/oaisis3"
entries_o3 = walk_files(root_o3)
print(f"  Found {len(entries_o3)} files in directory tree")

count_before = len(all_rows)
for e in entries_o3:
    row = parse_oasis3_entry(e)
    if row is not None:
        all_rows.append(row)

print(f"  Parsed {len(all_rows) - count_before} valid scans")

# ============================================================================
# BUILD DATAFRAME & QC
# ============================================================================

print("\n" + "=" * 70)
print("BUILDING INDEX DATAFRAME")
print("=" * 70)

nifti_index = pd.DataFrame(all_rows)

print(f"\n✓ Total scans indexed: {len(nifti_index)}")

print("\n--- Distribution by Dataset ---")
print(nifti_index["dataset"].value_counts())

print("\n--- Distribution by Source ---")
print(nifti_index["source"].value_counts())

print("\n--- T1-weighted Scans by Dataset ---")
t1w_counts = nifti_index.groupby(["dataset", "is_t1w"]).size().unstack(fill_value=0)
print(t1w_counts)

print("\n--- File Extensions by Dataset ---")
ext_counts = nifti_index.groupby(["dataset", "ext"]).size().unstack(fill_value=0)
print(ext_counts)

# ============================================================================
# VALIDATION CHECKS
# ============================================================================

print("\n" + "=" * 70)
print("VALIDATION CHECKS")
print("=" * 70)

# Check 1: No missing critical fields
assert nifti_index["subject_id"].notnull().all(), "❌ Missing subject_id values detected"
print("✓ No missing subject_id values")

assert nifti_index["visit_code"].notnull().all(), "❌ Missing visit_code values detected"
print("✓ No missing visit_code values")

assert nifti_index["dataset"].notnull().all(), "❌ Missing dataset values detected"
print("✓ No missing dataset values")

# Check 2: scan_uid uniqueness
assert nifti_index["scan_uid"].is_unique, "❌ Duplicate scan_uid detected"
print("✓ All scan_uid values are unique")

# Check 3: Expected dataset values
valid_datasets = {"OASIS2", "OASIS3"}
assert set(nifti_index["dataset"].unique()).issubset(valid_datasets), "❌ Unexpected dataset values"
print(f"✓ Dataset values are valid: {nifti_index['dataset'].unique()}")

# Check 4: Subject ID format validation
oasis2_pattern = re.compile(r'^OAS2_\d{4}$')
oasis3_pattern = re.compile(r'^OAS3\d{4}$')

oasis2_subjects = nifti_index[nifti_index["dataset"] == "OASIS2"]["subject_id"].unique()
oasis3_subjects = nifti_index[nifti_index["dataset"] == "OASIS3"]["subject_id"].unique()

invalid_o2 = [s for s in oasis2_subjects if not oasis2_pattern.match(s)]
invalid_o3 = [s for s in oasis3_subjects if not oasis3_pattern.match(s)]

if invalid_o2:
    print(f"⚠️  Warning: {len(invalid_o2)} OASIS-2 subjects with unexpected ID format")
    print(f"   Examples: {invalid_o2[:3]}")
else:
    print(f"✓ All {len(oasis2_subjects)} OASIS-2 subject IDs match expected format")

if invalid_o3:
    print(f"⚠️  Warning: {len(invalid_o3)} OASIS-3 subjects with unexpected ID format")
    print(f"   Examples: {invalid_o3[:3]}")
else:
    print(f"✓ All {len(oasis3_subjects)} OASIS-3 subject IDs match expected format")

# Check 5: T1w scan availability
n_t1w_o2 = nifti_index[(nifti_index["dataset"] == "OASIS2") & (nifti_index["is_t1w"])].shape[0]
n_t1w_o3 = nifti_index[(nifti_index["dataset"] == "OASIS3") & (nifti_index["is_t1w"])].shape[0]

print(f"\n✓ OASIS-2 T1w scans: {n_t1w_o2}")
print(f"✓ OASIS-3 T1w scans: {n_t1w_o3}")

assert n_t1w_o2 > 0, "❌ No T1w scans found in OASIS-2"
assert n_t1w_o3 > 0, "❌ No T1w scans found in OASIS-3"

# ============================================================================
# SAVE OUTPUT
# ============================================================================

output_path = "nifti_index_parsed.csv"
nifti_index.to_csv(output_path, index=False)

print("\n" + "=" * 70)
print(f"✓ SAVED: {output_path}")
print("=" * 70)

print("\n--- Preview (first 10 rows) ---")
print(nifti_index.head(10).to_string())

print("\n--- Sample OASIS-2 Entry ---")
sample_o2 = nifti_index[nifti_index["dataset"] == "OASIS2"].iloc[0]
print(f"  Subject: {sample_o2['subject_id']}, Visit: {sample_o2['visit_code']}, Run: {sample_o2['run_id']}")
print(f"  File: {sample_o2['filename']}")
print(f"  Path: {sample_o2['nifti_path']}")

print("\n--- Sample OASIS-3 Entry ---")
sample_o3 = nifti_index[nifti_index["dataset"] == "OASIS3"].iloc[0]
print(f"  Subject: {sample_o3['subject_id']}, Visit: {sample_o3['visit_code']}, Run: {sample_o3['run_id']}")
print(f"  File: {sample_o3['filename']}")
print(f"  Path: {sample_o3['nifti_path']}")

print("\n" + "=" * 70)
print("SNIPPET S0: COMPLETE ✓")
print("=" * 70)

SNIPPET S0: RAW SCAN INDEX BUILDER

[1/3] Scanning OASIS-2 PART 1...
  Found 1849 files in directory tree
  Parsed 772 valid scans

[2/3] Scanning OASIS-2 PART 2...
  Found 1526 files in directory tree
  Parsed 596 valid scans

[3/3] Scanning OASIS-3...
  Found 635 files in directory tree
  Parsed 635 valid scans

BUILDING INDEX DATAFRAME

✓ Total scans indexed: 2003

--- Distribution by Dataset ---
dataset
OASIS2    1368
OASIS3     635
Name: count, dtype: int64

--- Distribution by Source ---
source
OASIS2_P1    772
OASIS3       635
OASIS2_P2    596
Name: count, dtype: int64

--- T1-weighted Scans by Dataset ---
is_t1w   False  True 
dataset              
OASIS2       1   1367
OASIS3       0    635

--- File Extensions by Dataset ---
ext      .hdr  .nii
dataset            
OASIS2   1368     0
OASIS3      0   635

VALIDATION CHECKS
✓ No missing subject_id values
✓ No missing visit_code values
✓ No missing dataset values
✓ All scan_uid values are unique
✓ Dataset values are valid: ['OAS

In [2]:
# ============================================================================
# SNIPPET S1b: DATA AUDIT & SUBJECT/VISIT TABLES (REVISED OASIS-3 MATCHING)
# ============================================================================
# Builds visit-level and subject-level tables with clinical metadata.
# Uses nearest-day matching for OASIS-3 to handle asynchronous MRI/clinical visits.

import os
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("SNIPPET S1b: DATA AUDIT (REVISED OASIS-3 NEAREST-DAY MATCHING)")
print("=" * 80)

# ============================================================================
# STEP A: ONE T1 PER VISIT (VISIT-LEVEL INDEX)
# ============================================================================

print("\n[STEP A] Building visit-level T1 index from scan index...")

scan_index = pd.read_csv("nifti_index_parsed.csv")
print(f"  Loaded scan index: {len(scan_index)} total scans")

# Filter to T1-weighted scans only
scan_index = scan_index[scan_index["is_t1w"] == True].copy()
print(f"  After T1w filter: {len(scan_index)} scans")

# Group by (dataset, subject_id, visit_code) and pick one representative
group_cols = ["dataset", "subject_id", "visit_code"]
visit_rows = []

for (ds, sid, vid), grp in scan_index.groupby(group_cols):
    # Pick scan with smallest run_id (tie-break by first occurrence)
    rep = grp.sort_values("run_id").iloc[0]
    visit_rows.append({
        "dataset": ds,
        "subject_id": sid,
        "visit_code": vid,
        "scan_uid": rep["scan_uid"],
        "nifti_path": rep["nifti_path"],
        "run_id": rep["run_id"],
    })

visit_level_index = pd.DataFrame(visit_rows)
print(f"  ✓ Created visit-level index: {len(visit_level_index)} unique visits")
print(f"    OASIS2: {(visit_level_index['dataset'] == 'OASIS2').sum()} visits")
print(f"    OASIS3: {(visit_level_index['dataset'] == 'OASIS3').sum()} visits")

# ============================================================================
# STEP B: LOAD & MAP OASIS-2 CLINICAL DATA (UNCHANGED)
# ============================================================================

print("\n[STEP B] Processing OASIS-2 clinical data...")

# Load OASIS-2 clinical data
o2_clin_path = "/kaggle/input/mri-and-alzheimers/oasis_longitudinal.csv"
df_o2_raw = pd.read_csv(o2_clin_path)

print(f"  OASIS-2 clinical shape: {df_o2_raw.shape}")
print(f"  Columns: {list(df_o2_raw.columns)}")

# Map OASIS-2 IDs and visits
df_o2_clin = pd.DataFrame()

# Subject ID mapping
if 'Subject ID' in df_o2_raw.columns:
    df_o2_clin["subject_id"] = df_o2_raw['Subject ID'].str.strip()
elif 'ID' in df_o2_raw.columns:
    df_o2_clin["subject_id"] = df_o2_raw['ID'].str.strip()
else:
    raise ValueError("Cannot find subject ID column in OASIS-2 clinical CSV")

# Visit code mapping
if 'Visit' in df_o2_raw.columns:
    df_o2_clin["visit_code"] = df_o2_raw['Visit'].apply(lambda x: f"MR{int(x)}" if pd.notnull(x) else np.nan)
elif 'MR ID' in df_o2_raw.columns:
    df_o2_clin["visit_code"] = df_o2_raw['MR ID'].apply(lambda x: f"MR{int(x)}" if pd.notnull(x) else np.nan)
else:
    raise ValueError("Cannot find visit column in OASIS-2 clinical CSV")

# CDR and MMSE
cdr_col = 'CDR' if 'CDR' in df_o2_raw.columns else [c for c in df_o2_raw.columns if 'cdr' in c.lower()][0]
mmse_col = 'MMSE' if 'MMSE' in df_o2_raw.columns else [c for c in df_o2_raw.columns if 'mmse' in c.lower()][0]

df_o2_clin["cdr"] = pd.to_numeric(df_o2_raw[cdr_col], errors='coerce')
df_o2_clin["mmse"] = pd.to_numeric(df_o2_raw[mmse_col], errors='coerce')

# Demographics
if 'M/F' in df_o2_raw.columns:
    df_o2_clin["sex"] = df_o2_raw['M/F']
elif 'Sex' in df_o2_raw.columns:
    df_o2_clin["sex"] = df_o2_raw['Sex']
else:
    df_o2_clin["sex"] = np.nan

if 'Age' in df_o2_raw.columns:
    df_o2_clin["age"] = pd.to_numeric(df_o2_raw['Age'], errors='coerce')
else:
    df_o2_clin["age"] = np.nan

print(f"  ✓ OASIS-2 clinical processed: {len(df_o2_clin)} rows")
print(f"    With CDR: {df_o2_clin['cdr'].notnull().sum()}")
print(f"    With MMSE: {df_o2_clin['mmse'].notnull().sum()}")

# Merge OASIS-2
vl_o2 = visit_level_index[visit_level_index["dataset"] == "OASIS2"].copy()
visits_o2 = vl_o2.merge(df_o2_clin, on=["subject_id", "visit_code"], how="left")
visits_o2["site"] = "OASIS2"

n_o2_matched = visits_o2['cdr'].notnull().sum()
print(f"  OASIS-2 merge: {n_o2_matched}/{len(vl_o2)} visits matched ({100*n_o2_matched/len(vl_o2):.1f}%)")

# ============================================================================
# STEP C: OASIS-3 NEAREST-DAY MATCHING (REVISED)
# ============================================================================

print("\n[STEP C] Processing OASIS-3 clinical data with nearest-day matching...")

# Load OASIS-3 clinical data
o3_clin_path = "/kaggle/input/oaisis-3-longitiudinal/oaisis3longitiudinal.csv"
df_o3_raw = pd.read_csv(o3_clin_path)

print(f"  OASIS-3 clinical shape: {df_o3_raw.shape}")
print(f"  Columns: {list(df_o3_raw.columns)}")

# Build OASIS-3 clinical table
df_o3_clin = pd.DataFrame()

# Subject ID - find the right column
id_candidates = ['ADRC_ADRCCLINICALDATA ID', 'Subject', 'OASISID', 'subject', 'ID']
subject_col = None
for col in id_candidates:
    if col in df_o3_raw.columns:
        subject_col = col
        break

if subject_col is None:
    # Try case-insensitive search
    for col in df_o3_raw.columns:
        if 'subject' in col.lower() or 'oasis' in col.lower():
            subject_col = col
            break

if subject_col is None:
    raise ValueError("Cannot find subject ID column in OASIS-3 clinical CSV")

print(f"  Using '{subject_col}' for subject_id")
df_o3_clin["subject_id"] = df_o3_raw[subject_col].astype(str).str.strip()

# Standardize subject IDs to OAS3XXXX format
def standardize_o3_id(raw_id):
    if pd.isna(raw_id) or raw_id == 'nan':
        return np.nan
    # If already formatted
    if re.match(r'^OAS3\d{4}$', raw_id):
        return raw_id
    # Extract digits
    digits = re.findall(r'\d+', raw_id)
    if digits:
        num = int(digits[0])
        return f"OAS3{num:04d}"
    return raw_id

df_o3_clin["subject_id"] = df_o3_clin["subject_id"].apply(standardize_o3_id)

# Days from baseline
days_candidates = ['days_to_visit', 'DAYS', 'days', 'Days']
days_col = None
for col in days_candidates:
    if col in df_o3_raw.columns:
        days_col = col
        break

if days_col is None:
    raise ValueError("Cannot find days column in OASIS-3 clinical CSV")

print(f"  Using '{days_col}' for days")
df_o3_clin["days"] = pd.to_numeric(df_o3_raw[days_col], errors='coerce')

# CDR - try both global (CDRTOT) and sum of boxes (CDRSUM)
cdr_candidates = ['CDRTOT', 'cdr', 'CDR', 'CDRSUM']
cdr_col = None
for col in cdr_candidates:
    if col in df_o3_raw.columns:
        cdr_col = col
        print(f"  Using '{cdr_col}' for CDR")
        break

if cdr_col:
    df_o3_clin["cdr_global"] = pd.to_numeric(df_o3_raw[cdr_col], errors='coerce')
else:
    print("  WARNING: No CDR column found")
    df_o3_clin["cdr_global"] = np.nan

# MMSE
mmse_candidates = ['MMSE', 'mmse']
mmse_col = None
for col in mmse_candidates:
    if col in df_o3_raw.columns:
        mmse_col = col
        print(f"  Using '{mmse_col}' for MMSE")
        break

if mmse_col:
    df_o3_clin["mmse"] = pd.to_numeric(df_o3_raw[mmse_col], errors='coerce')
else:
    print("  WARNING: No MMSE column found")
    df_o3_clin["mmse"] = np.nan

# Age
age_candidates = ['age at visit', 'age_at_entry', 'Age', 'age']
age_col = None
for col in age_candidates:
    if col in df_o3_raw.columns:
        age_col = col
        break

if age_col:
    df_o3_clin["age"] = pd.to_numeric(df_o3_raw[age_col], errors='coerce')
else:
    df_o3_clin["age"] = np.nan

# Sex
sex_candidates = ['sex', 'Sex', 'M/F']
sex_col = None
for col in sex_candidates:
    if col in df_o3_raw.columns:
        sex_col = col
        break

if sex_col:
    df_o3_clin["sex"] = df_o3_raw[sex_col]
else:
    df_o3_clin["sex"] = np.nan

# Clean clinical table
# Keep only rows with days information
df_o3_clin = df_o3_clin[df_o3_clin["days"].notnull()].copy()

# Keep only rows with at least CDR or MMSE
df_o3_clin = df_o3_clin[
    (df_o3_clin["cdr_global"].notnull()) | 
    (df_o3_clin["mmse"].notnull())
].copy()

print(f"  ✓ OASIS-3 clinical usable rows: {len(df_o3_clin)}")
print(f"    With CDR: {df_o3_clin['cdr_global'].notnull().sum()}")
print(f"    With MMSE: {df_o3_clin['mmse'].notnull().sum()}")
print(f"    Unique subjects: {df_o3_clin['subject_id'].nunique()}")

# Nearest-day matching
print("\n  Performing nearest-day matching for OASIS-3 MRI visits...")

vl_o3 = visit_level_index[visit_level_index["dataset"] == "OASIS3"].copy()
MAX_DAY_DIFF = 365  # Maximum allowed difference in days

o3_rows = []

for idx, row in vl_o3.iterrows():
    sid = row["subject_id"]
    vcode = row["visit_code"]  # e.g., "d0056"
    
    # Parse MRI days from visit code
    try:
        mri_days = int(vcode[1:])  # Strip 'd' and parse
    except:
        mri_days = np.nan
    
    # Initialize as NaN
    cdr = np.nan
    mmse = np.nan
    age = np.nan
    sex = np.nan
    day_diff = np.nan
    
    # Find clinical visits for this subject
    subj_clin = df_o3_clin[df_o3_clin["subject_id"] == sid]
    
    if len(subj_clin) > 0 and pd.notnull(mri_days):
        # Compute absolute day differences
        diffs = (subj_clin["days"] - mri_days).abs()
        best_idx = diffs.idxmin()
        best = subj_clin.loc[best_idx]
        day_diff = diffs.loc[best_idx]
        
        # Only assign if within acceptable window
        if day_diff <= MAX_DAY_DIFF:
            cdr = best["cdr_global"]
            mmse = best["mmse"]
            age = best["age"]
            sex = best["sex"]
    
    o3_rows.append({
        "dataset": row["dataset"],
        "subject_id": sid,
        "visit_code": vcode,
        "scan_uid": row["scan_uid"],
        "nifti_path": row["nifti_path"],
        "run_id": row["run_id"],
        "cdr": cdr,
        "mmse": mmse,
        "age": age,
        "sex": sex,
        "site": "OASIS3",
        "day_diff": day_diff,
    })

visits_o3 = pd.DataFrame(o3_rows)

print(f"  ✓ OASIS-3 MRI visits: {len(visits_o3)}")
print(f"    With non-null CDR: {visits_o3['cdr'].notnull().sum()}")
print(f"    With non-null MMSE: {visits_o3['mmse'].notnull().sum()}")
print(f"    With both CDR & MMSE: {(visits_o3['cdr'].notnull() & visits_o3['mmse'].notnull()).sum()}")

# Day difference statistics for labeled visits
labeled_o3 = visits_o3[visits_o3['cdr'].notnull()]
if len(labeled_o3) > 0:
    print(f"\n  Day difference statistics (for {len(labeled_o3)} labeled visits):")
    print(f"    Mean: {labeled_o3['day_diff'].mean():.1f} days")
    print(f"    Median: {labeled_o3['day_diff'].median():.1f} days")
    print(f"    Max: {labeled_o3['day_diff'].max():.1f} days")
    print(f"    Within 30 days: {(labeled_o3['day_diff'] <= 30).sum()} ({100*(labeled_o3['day_diff'] <= 30).sum()/len(labeled_o3):.1f}%)")
    print(f"    Within 90 days: {(labeled_o3['day_diff'] <= 90).sum()} ({100*(labeled_o3['day_diff'] <= 90).sum()/len(labeled_o3):.1f}%)")

# ============================================================================
# STEP D: COMBINE DATASETS
# ============================================================================

print("\n[STEP D] Combining OASIS-2 and OASIS-3 visits...")

# Ensure both have same columns
visits_o2["day_diff"] = np.nan  # OASIS-2 doesn't use nearest-day matching

visits_all = pd.concat([visits_o2, visits_o3], ignore_index=True)

print(f"  ✓ Combined visits: {len(visits_all)}")
print(f"    OASIS2: {len(visits_o2)}")
print(f"    OASIS3: {len(visits_o3)}")

# ============================================================================
# STEP E: VISIT-LEVEL DIAGNOSIS & INCLUSION CRITERIA
# ============================================================================

print("\n[STEP E] Defining visit-level diagnosis and inclusion criteria...")

# Initialize diagnosis_visit as NaN
visits_all["diagnosis_visit"] = np.nan

# Define CN (CDR=0) and AD (CDR≥1)
mask_CN = (visits_all["cdr"] == 0.0)
mask_AD = (visits_all["cdr"] >= 1.0)

visits_all.loc[mask_CN, "diagnosis_visit"] = 0
visits_all.loc[mask_AD, "diagnosis_visit"] = 1

# Has MMSE flag
visits_all["has_mmse"] = visits_all["mmse"].notnull()

# Inclusion criteria
visits_all["include_for_classifier"] = (
    visits_all["diagnosis_visit"].isin([0, 1]) &
    visits_all["has_mmse"] &
    visits_all["nifti_path"].notnull()
)

print("\n--- Diagnosis Distribution (Visit-Level) ---")
diag_counts = visits_all["diagnosis_visit"].value_counts(dropna=False).sort_index()
print(diag_counts)
if 0.0 in diag_counts.index:
    print(f"  CN (0): {int(diag_counts[0.0])}")
if 1.0 in diag_counts.index:
    print(f"  AD (1): {int(diag_counts[1.0])}")
nan_count = visits_all["diagnosis_visit"].isna().sum()
print(f"  Excluded (NaN): {nan_count}")

print(f"\n✓ Visits eligible for classifier: {visits_all['include_for_classifier'].sum()}")

print("\n--- Eligible Visits by Dataset ---")
eligible_by_dataset = visits_all[visits_all["include_for_classifier"]].groupby("dataset")["diagnosis_visit"].value_counts().unstack(fill_value=0)
print(eligible_by_dataset)

# ============================================================================
# STEP F: SUBJECT-LEVEL AGGREGATION
# ============================================================================

print("\n[STEP F] Aggregating to subject-level table...")

subj_rows = []

for (ds, sid), grp in visits_all.groupby(["dataset", "subject_id"]):
    n_total = len(grp)
    n_eligible = grp["include_for_classifier"].sum()
    n_cn = (grp["diagnosis_visit"] == 0).sum()
    n_ad = (grp["diagnosis_visit"] == 1).sum()
    
    has_any_ad = (n_ad > 0)
    has_only_cn = (n_ad == 0) and (n_cn > 0)
    
    if has_any_ad:
        diag_subj = 1
    elif has_only_cn:
        diag_subj = 0
    else:
        diag_subj = np.nan
    
    include_for_splits = (diag_subj in [0, 1]) and (n_eligible >= 1)
    
    subj_rows.append({
        "dataset": ds,
        "subject_id": sid,
        "diagnosis_subject": diag_subj,
        "n_visits_total": n_total,
        "n_visits_class_eligible": n_eligible,
        "n_visits_CN": n_cn,
        "n_visits_AD": n_ad,
        "include_for_splits": include_for_splits,
    })

subjects_table = pd.DataFrame(subj_rows)

print(f"  ✓ Subject-level table: {len(subjects_table)} subjects")

print("\n--- Subjects Eligible for Splits ---")
eligible_subj = subjects_table[subjects_table["include_for_splits"]]
print(f"  Total eligible: {len(eligible_subj)}")
print(f"\n  By dataset:")
dataset_subj = eligible_subj.groupby("dataset")["subject_id"].count()
print(dataset_subj)
print(f"\n  By diagnosis:")
diag_subj = eligible_subj["diagnosis_subject"].value_counts().sort_index()
print(f"    CN subjects (0): {int(diag_subj.get(0, 0))}")
print(f"    AD subjects (1): {int(diag_subj.get(1, 0))}")

# ============================================================================
# STEP G: BASELINE ACCURACY & VALIDATION
# ============================================================================

print("\n[STEP G] Computing baseline accuracy and validation...")

eligible_visits = visits_all[visits_all["include_for_classifier"]].copy()

num_CN = (eligible_visits["diagnosis_visit"] == 0).sum()
num_AD = (eligible_visits["diagnosis_visit"] == 1).sum()
total_eligible = num_CN + num_AD

if total_eligible > 0:
    acc_baseline = num_CN / total_eligible
    
    print(f"\n{'='*80}")
    print("ALWAYS-CN BASELINE")
    print(f"{'='*80}")
    print(f"  CN visits: {num_CN}")
    print(f"  AD visits: {num_AD}")
    print(f"  Total eligible: {total_eligible}")
    print(f"  Class ratio (CN:AD): {num_CN/num_AD:.2f}:1" if num_AD > 0 else "  Class ratio: undefined (no AD)")
    print(f"  Always-CN baseline ACC: {acc_baseline:.4f} ({100*acc_baseline:.2f}%)")
    print(f"{'='*80}")
else:
    print("WARNING: No eligible visits for baseline calculation")

# Validation checks
print("\n--- Validation Checks ---")

# Check 1: No duplicate visits
dup_check = visits_all[["dataset", "subject_id", "visit_code"]].duplicated().any()
assert not dup_check, "❌ Duplicate visits detected"
print("✓ No duplicate (dataset, subject_id, visit_code) combinations")

# Check 2: Eligible visits have required fields
if len(eligible_visits) > 0:
    assert eligible_visits["cdr"].notnull().all(), "❌ Eligible visits with null CDR"
    assert eligible_visits["mmse"].notnull().all(), "❌ Eligible visits with null MMSE"
    assert eligible_visits["nifti_path"].notnull().all(), "❌ Eligible visits with null nifti_path"
    print("✓ All eligible visits have CDR, MMSE, and nifti_path")

# ============================================================================
# SAVE OUTPUTS
# ============================================================================

print("\n" + "=" * 80)
print("SAVING OUTPUTS")
print("=" * 80)

visits_all.to_csv("visits_table.csv", index=False)
print("✓ Saved: visits_table.csv")
print(f"  Shape: {visits_all.shape}")

subjects_table.to_csv("subjects_table.csv", index=False)
print("✓ Saved: subjects_table.csv")
print(f"  Shape: {subjects_table.shape}")

print("\n" + "=" * 80)
print("SNIPPET S1b: COMPLETE ✓")
print("=" * 80)


SNIPPET S1b: DATA AUDIT (REVISED OASIS-3 NEAREST-DAY MATCHING)

[STEP A] Building visit-level T1 index from scan index...
  Loaded scan index: 2003 total scans
  After T1w filter: 2002 scans
  ✓ Created visit-level index: 796 unique visits
    OASIS2: 373 visits
    OASIS3: 423 visits

[STEP B] Processing OASIS-2 clinical data...
  OASIS-2 clinical shape: (373, 15)
  Columns: ['Subject ID', 'MRI ID', 'Group', 'Visit', 'MR Delay', 'M/F', 'Hand', 'Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF']
  ✓ OASIS-2 clinical processed: 373 rows
    With CDR: 373
    With MMSE: 371
  OASIS-2 merge: 373/373 visits matched (100.0%)

[STEP C] Processing OASIS-3 clinical data with nearest-day matching...
  OASIS-3 clinical shape: (8626, 23)
  Columns: ['OASISID', 'OASIS_session_label', 'days_to_visit', 'age at visit', 'MMSE', 'memory', 'orient', 'judgment', 'commun', 'homehobb', 'perscare', 'CDRSUM', 'CDRTOT', 'dx1_code', 'dx2_code', 'dx3_code', 'dx4_code', 'dx5_code', 'dx1', 'dx2', 'dx3', '

In [3]:
# ============================================================================
# SNIPPET S2: 3D PREPROCESSING PIPELINE (COMPLETE - 4D HANDLING + OASIS-3 FIX)
# ============================================================================
# Registers raw T1 volumes to MNI space, applies brain mask, and performs
# robust percentile intensity normalization for multi-site standardization

import os
import numpy as np
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
from pathlib import Path
from tqdm import tqdm
import warnings
import shutil
import hashlib
warnings.filterwarnings('ignore')

# Nilearn for automatic template downloading
from nilearn.datasets import load_mni152_template, load_mni152_brain_mask

print("=" * 80)
print("SNIPPET S2: 3D PREPROCESSING PIPELINE (4D HANDLING + OASIS-3 FIXED)")
print("=" * 80)

# ============================================================================
# CONFIGURATION
# ============================================================================

OUTPUT_ROOT = "processed_mri"
os.makedirs(OUTPUT_ROOT, exist_ok=True)

# Registration parameters
RIGID_DOF = 6
MAX_ITER = 200
MNI_RESOLUTION = 1  # 1mm isotropic

# Temporary directory for cleaned files (both datasets)
TEMP_DIR = "temp_cleaned_volumes"
os.makedirs(TEMP_DIR, exist_ok=True)

print(f"\nConfiguration:")
print(f"  Output root: {OUTPUT_ROOT}")
print(f"  MNI resolution: {MNI_RESOLUTION}mm isotropic")
print(f"  Registration DOF: {RIGID_DOF} (rigid)")
print(f"  Max iterations: {MAX_ITER}")
print(f"  Temp directory: {TEMP_DIR}")

# ============================================================================
# STEP 0: LOAD VISITS TABLE & FILTER
# ============================================================================

print("\n[STEP 0] Loading visits table...")

visits = pd.read_csv("visits_table.csv")
print(f"  Total visits in table: {len(visits)}")

visits_proc = visits[visits["include_for_classifier"] == True].copy()
print(f"  Visits to preprocess: {len(visits_proc)}")

print("\n--- Breakdown by Dataset & Diagnosis ---")
breakdown = visits_proc.groupby(["dataset", "diagnosis_visit"]).size().unstack(fill_value=0)
print(breakdown)

# ============================================================================
# STEP 1: LOAD MNI TEMPLATE & BRAIN MASK (NILEARN)
# ============================================================================

print("\n[STEP 1] Loading MNI template and brain mask via nilearn...")
print("  (First run will download and cache ~50MB, subsequent runs are instant)")

# Load MNI152 template (1mm resolution, skull-stripped)
print("\n  Loading MNI152 T1 template...")
mni_template_nib = load_mni152_template(resolution=MNI_RESOLUTION)
mni_template_data = mni_template_nib.get_fdata()
mni_affine = mni_template_nib.affine
mni_shape = mni_template_data.shape

print(f"  ✓ MNI template loaded")
print(f"    Shape: {mni_shape}")
print(f"    Voxel size: {mni_template_nib.header.get_zooms()[:3]} mm")
print(f"    Data range: [{mni_template_data.min():.1f}, {mni_template_data.max():.1f}]")

# Load brain mask
print("\n  Loading MNI152 brain mask...")
brain_mask_nib = load_mni152_brain_mask(resolution=MNI_RESOLUTION, threshold=0.2)
brain_mask = brain_mask_nib.get_fdata().astype(bool)
brain_voxel_count_ref = int(brain_mask.sum())

assert brain_mask.shape == mni_shape, f"Mask shape {brain_mask.shape} != template shape {mni_shape}"

print(f"  ✓ Brain mask loaded")
print(f"    Shape: {brain_mask.shape}")
print(f"    Brain voxels: {brain_voxel_count_ref:,}")
print(f"    Brain fraction: {100 * brain_voxel_count_ref / np.prod(mni_shape):.1f}%")

# Save template and mask to disk for SimpleITK registration
print("\n  Saving template to disk for SimpleITK...")
TEMPLATE_DIR = "mni_template"
os.makedirs(TEMPLATE_DIR, exist_ok=True)

MNI_TEMPLATE_PATH = os.path.join(TEMPLATE_DIR, "mni_icbm152_t1_1mm.nii.gz")
MNI_BRAIN_MASK_PATH = os.path.join(TEMPLATE_DIR, "mni_icbm152_t1_1mm_brain_mask.nii.gz")

if not os.path.exists(MNI_TEMPLATE_PATH):
    nib.save(mni_template_nib, MNI_TEMPLATE_PATH)
    print(f"    ✓ Saved template: {MNI_TEMPLATE_PATH}")
else:
    print(f"    ✓ Template already exists: {MNI_TEMPLATE_PATH}")

if not os.path.exists(MNI_BRAIN_MASK_PATH):
    nib.save(brain_mask_nib, MNI_BRAIN_MASK_PATH)
    print(f"    ✓ Saved mask: {MNI_BRAIN_MASK_PATH}")
else:
    print(f"    ✓ Mask already exists: {MNI_BRAIN_MASK_PATH}")

# Convert to SimpleITK for registration
mni_template_sitk = sitk.ReadImage(MNI_TEMPLATE_PATH)

# ============================================================================
# FILE HANDLING FUNCTIONS
# ============================================================================

def find_actual_oasis3_file(raw_path):
    """
    OASIS-3 files are in nested directories. Find the actual .nii file.
    
    Example structure:
      /path/to/sub-OAS30001_ses-d0129_run-01_T1w.nii/  <- directory
        sub-OAS30001_sess-d0129_run-01_T1w.nii        <- actual file (note: "sess" not "ses")
    
    Args:
        raw_path: Path from nifti_index_parsed.csv
    
    Returns:
        Path to actual .nii file
    """
    path_obj = Path(raw_path)
    
    # Case 1: Path points to a directory, look inside for .nii file
    if path_obj.is_dir():
        nii_files = list(path_obj.glob("*.nii"))
        if nii_files:
            # Prefer T1w files
            for f in nii_files:
                if "T1w" in f.name or "t1" in f.name.lower():
                    return str(f)
            return str(nii_files[0])
    
    # Case 2: Path itself is the file
    if path_obj.exists() and path_obj.is_file():
        return str(path_obj)
    
    # Case 3: Path doesn't exist, try looking in parent directory
    if not path_obj.exists():
        parent = path_obj.parent
        if parent.exists() and parent.is_dir():
            nii_files = list(parent.glob("*.nii"))
            if nii_files:
                # Find file with most similar name
                for f in nii_files:
                    if "T1w" in f.name or "t1" in f.name.lower():
                        return str(f)
                return str(nii_files[0])
    
    # Case 4: Look for compressed version
    gz_path = Path(str(path_obj) + ".gz")
    if gz_path.exists():
        return str(gz_path)
    
    raise FileNotFoundError(f"Cannot find actual NIfTI file for: {raw_path}")


def collapse_4d_to_3d(data):
    """
    Collapse 4D volume to 3D.
    
    Common cases:
    - (X, Y, Z, 1): Singleton 4th dimension → squeeze
    - (X, Y, Z, T): Multiple timepoints/echoes → average or take first
    
    Args:
        data: numpy array, possibly 4D
    
    Returns:
        3D numpy array
    """
    if data.ndim == 3:
        return data
    
    if data.ndim == 4:
        if data.shape[-1] == 1:
            # Singleton dimension: (X, Y, Z, 1) → (X, Y, Z)
            return data[..., 0]
        else:
            # Multiple volumes: average over last axis
            # Alternative: data[..., 0] to take first volume
            return np.mean(data, axis=-1)
    
    # Fallback: try to squeeze all singleton dimensions
    data_squeezed = np.squeeze(data)
    if data_squeezed.ndim == 3:
        return data_squeezed
    
    raise ValueError(f"Cannot collapse {data.ndim}D volume with shape {data.shape} to 3D")


def prepare_volume_for_sitk(raw_path, dataset, temp_dir=TEMP_DIR):
    """
    Load raw volume with nibabel, handle 4D→3D, clean headers, save for SimpleITK.
    
    This unified function handles both OASIS-2 and OASIS-3:
    - OASIS-2: May be 4D with shape (X,Y,Z,1) or (X,Y,Z,T)
    - OASIS-3: Nested directories + BIDS headers
    
    Args:
        raw_path: Original path from dataset
        dataset: "OASIS2" or "OASIS3"
        temp_dir: Directory for temporary cleaned files
    
    Returns:
        Path to cleaned 3D volume ready for SimpleITK
    """
    # Find actual file (especially important for OASIS-3)
    if dataset == "OASIS3":
        actual_file = find_actual_oasis3_file(raw_path)
    else:
        actual_file = raw_path
    
    # Load with nibabel
    try:
        img = nib.load(actual_file)
        data = img.get_fdata()
        affine = img.affine
    except Exception as e:
        raise ValueError(f"Nibabel load failed: {e}")
    
    # Collapse 4D → 3D if needed
    try:
        data_3d = collapse_4d_to_3d(data)
    except Exception as e:
        raise ValueError(f"Failed to collapse to 3D: {e}")
    
    # Validate 3D data
    if data_3d.size == 0:
        raise ValueError("Empty data array")
    
    if np.all(data_3d == 0):
        raise ValueError("All-zero volume")
    
    if data_3d.ndim != 3:
        raise ValueError(f"Expected 3D after collapsing, got {data_3d.ndim}D with shape {data_3d.shape}")
    
    # Create cleaned image with minimal header
    img_clean = nib.Nifti1Image(data_3d.astype(np.float32), affine)
    
    # Generate unique temp filename
    hash_suffix = hashlib.md5(actual_file.encode()).hexdigest()[:8]
    temp_filename = f"{dataset}_{hash_suffix}_cleaned.nii.gz"
    temp_path = os.path.join(temp_dir, temp_filename)
    
    # Save cleaned version
    nib.save(img_clean, temp_path)
    
    return temp_path


# ============================================================================
# PREPROCESSING FUNCTIONS
# ============================================================================

def rigid_register_to_mni(moving_path, fixed_sitk):
    """
    Rigidly register moving image to MNI template using mutual information.
    
    Args:
        moving_path: Path to cleaned 3D NIfTI file
        fixed_sitk: SimpleITK image of MNI template
    
    Returns:
        registered_sitk: Registered SimpleITK image in MNI space
        final_transform: The computed rigid transform
    """
    # Load moving image
    try:
        moving_sitk = sitk.ReadImage(str(moving_path))
    except Exception as e:
        raise ValueError(f"SimpleITK cannot read file: {e}")
    
    # Validate moving image
    if moving_sitk.GetSize()[0] == 0:
        raise ValueError("Moving image has zero size")
    
    # Initialize registration method
    registration = sitk.ImageRegistrationMethod()
    
    # Similarity metric: Mattes Mutual Information
    registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration.SetMetricSamplingStrategy(registration.RANDOM)
    registration.SetMetricSamplingPercentage(0.01)
    
    # Interpolator
    registration.SetInterpolator(sitk.sitkLinear)
    
    # Optimizer: Gradient Descent
    registration.SetOptimizerAsGradientDescent(
        learningRate=1.0,
        numberOfIterations=MAX_ITER,
        convergenceMinimumValue=1e-6,
        convergenceWindowSize=10
    )
    registration.SetOptimizerScalesFromPhysicalShift()
    
    # Setup for rigid transform (3 rotations + 3 translations)
    try:
        initial_transform = sitk.CenteredTransformInitializer(
            fixed_sitk,
            moving_sitk,
            sitk.Euler3DTransform(),
            sitk.CenteredTransformInitializerFilter.GEOMETRY
        )
    except Exception as e:
        raise ValueError(f"Transform initialization failed: {e}")
    
    registration.SetInitialTransform(initial_transform, inPlace=False)
    
    # Execute registration
    try:
        final_transform = registration.Execute(fixed_sitk, moving_sitk)
    except Exception as e:
        raise ValueError(f"Registration execution failed: {e}")
    
    # Resample moving image to fixed image space
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed_sitk)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(0)
    resampler.SetTransform(final_transform)
    
    registered_sitk = resampler.Execute(moving_sitk)
    
    return registered_sitk, final_transform


def preprocess_visit(row, mni_template_sitk, brain_mask, mni_affine):
    """
    Preprocess a single visit: clean file, register to MNI, mask, normalize.
    
    Args:
        row: DataFrame row with visit metadata
        mni_template_sitk: SimpleITK MNI template
        brain_mask: Binary brain mask array (nibabel convention)
        mni_affine: Affine matrix for output NIfTI
    
    Returns:
        dict with preprocessing results
    """
    raw_path = row["nifti_path"]
    dataset = row["dataset"]
    subject_id = row["subject_id"]
    visit_code = row["visit_code"]
    
    # Step 1: Prepare cleaned 3D volume (handles both 4D OASIS-2 and nested OASIS-3)
    try:
        cleaned_path = prepare_volume_for_sitk(raw_path, dataset)
    except Exception as e:
        raise ValueError(f"Volume preparation failed: {e}")
    
    # Step 2: Rigid registration to MNI
    try:
        img_mni_sitk, transform = rigid_register_to_mni(cleaned_path, mni_template_sitk)
        
        # Convert SimpleITK to numpy array
        # SimpleITK uses (z, y, x) ordering, nibabel uses (x, y, z)
        data_mni = sitk.GetArrayFromImage(img_mni_sitk)
        data_mni = np.transpose(data_mni, (2, 1, 0))  # Convert to nibabel convention
        data_mni = data_mni.astype(np.float32)
        
    except Exception as e:
        raise ValueError(f"Registration failed: {e}")
    
    # Step 3: Apply brain mask and compute percentiles
    try:
        masked_vals = data_mni[brain_mask]
        
        if masked_vals.size == 0:
            raise ValueError("No brain voxels found after masking")
        
        # Robust percentile normalization on brain voxels only
        p1 = np.percentile(masked_vals, 1)
        p99 = np.percentile(masked_vals, 99)
        
        if (p99 - p1) <= 1e-6:
            raise ValueError(f"Invalid percentile range: p1={p1:.2f}, p99={p99:.2f}")
        
    except Exception as e:
        raise ValueError(f"Percentile computation failed: {e}")
    
    # Step 4: Clip and normalize to [0, 1]
    data_clipped = np.clip(data_mni, p1, p99)
    data_norm = (data_clipped - p1) / (p99 - p1)
    data_norm = np.clip(data_norm, 0.0, 1.0).astype(np.float32)
    
    # Step 5: Zero out non-brain voxels (skull suppression)
    data_norm[~brain_mask] = 0.0
    
    # Step 6: Save preprocessed volume
    out_dir = os.path.join(OUTPUT_ROOT, dataset, subject_id)
    os.makedirs(out_dir, exist_ok=True)
    
    out_name = f"{subject_id}_{visit_code}_MNI_norm.nii.gz"
    out_path = os.path.join(out_dir, out_name)
    
    # Create nibabel image with MNI affine
    img_out = nib.Nifti1Image(data_norm, mni_affine)
    nib.save(img_out, out_path)
    
    return {
        "proc_nifti_path": out_path,
        "p1": float(p1),
        "p99": float(p99),
        "brain_voxel_count": brain_voxel_count_ref,
        "preproc_ok": True,
        "error_msg": "",
    }


# ============================================================================
# STEP 2: PROCESS ALL VISITS
# ============================================================================

print("\n[STEP 2] Processing visits with 4D handling and OASIS-3 fixes...")
print("  ⏱️  Estimated time: 30-60 seconds per visit (~5-10 hours total)")
print("  💡 Tip: Kaggle will auto-save progress if notebook times out\n")

rows_out = []
n_success = 0
n_fail = 0
n_fail_o2 = 0
n_fail_o3 = 0

for idx, row in tqdm(visits_proc.iterrows(), total=len(visits_proc), desc="Preprocessing", ncols=100):
    try:
        res = preprocess_visit(row, mni_template_sitk, brain_mask, mni_affine)
        n_success += 1
        
    except Exception as e:
        n_fail += 1
        if row["dataset"] == "OASIS2":
            n_fail_o2 += 1
        else:
            n_fail_o3 += 1
        
        error_msg = str(e)[:200]
        
        # Log first 5 failures in detail
        if n_fail <= 5:
            print(f"\n⚠️  Preprocessing failed for {row['scan_uid']}")
            print(f"    Dataset: {row['dataset']}, Subject: {row['subject_id']}, Visit: {row['visit_code']}")
            print(f"    Raw path: {row['nifti_path']}")
            print(f"    Error: {error_msg}")
        
        res = {
            "proc_nifti_path": None,
            "p1": np.nan,
            "p99": np.nan,
            "brain_voxel_count": np.nan,
            "preproc_ok": False,
            "error_msg": error_msg,
        }
    
    # Combine row data with preprocessing results
    rows_out.append({
        "dataset": row["dataset"],
        "site": row["site"],
        "subject_id": row["subject_id"],
        "visit_code": row["visit_code"],
        "scan_uid": row["scan_uid"],
        "raw_nifti_path": row["nifti_path"],
        "diagnosis_visit": row["diagnosis_visit"],
        "include_for_classifier": row["include_for_classifier"],
        **res,
    })

visits_preproc = pd.DataFrame(rows_out)

print(f"\n✓ Preprocessing complete")
print(f"  Total successful: {n_success}/{len(visits_proc)} ({100*n_success/len(visits_proc):.1f}%)")
print(f"  OASIS2 failures: {n_fail_o2}/{(visits_proc['dataset']=='OASIS2').sum()}")
print(f"  OASIS3 failures: {n_fail_o3}/{(visits_proc['dataset']=='OASIS3').sum()}")

# ============================================================================
# STEP 3: QUALITY CONTROL STATISTICS
# ============================================================================

print("\n" + "=" * 80)
print("QUALITY CONTROL STATISTICS")
print("=" * 80)

ok_visits = visits_preproc[visits_preproc["preproc_ok"] == True]
print(f"\nPreprocessed visits (OK): {len(ok_visits)}/{len(visits_preproc)}")

if len(ok_visits) > 0:
    # Breakdown by dataset and diagnosis
    print("\n--- Successful Preprocessing by Dataset & Diagnosis ---")
    ok_breakdown = ok_visits.groupby(["dataset", "diagnosis_visit"]).size().unstack(fill_value=0)
    print(ok_breakdown)
    
    print("\n--- By Site & Diagnosis ---")
    site_breakdown = ok_visits.groupby(["site", "diagnosis_visit"]).size().unstack(fill_value=0)
    print(site_breakdown)
    
    # Percentile statistics
    print("\n--- Percentile Statistics (p1) ---")
    print(ok_visits["p1"].describe())
    
    print("\n--- Percentile Statistics (p99) ---")
    print(ok_visits["p99"].describe())
    
    # Site-wise comparison (check for scanner differences)
    print("\n--- Percentile Stats by Site (Multi-site Normalization Check) ---")
    site_stats = ok_visits.groupby("site")[["p1", "p99"]].agg(["mean", "median", "std"])
    print(site_stats)
    
    # Compute site differences
    sites = ok_visits["site"].unique()
    if "OASIS2" in sites and "OASIS3" in sites:
        o2_p1_mean = ok_visits[ok_visits["site"] == "OASIS2"]["p1"].mean()
        o3_p1_mean = ok_visits[ok_visits["site"] == "OASIS3"]["p1"].mean()
        o2_p99_mean = ok_visits[ok_visits["site"] == "OASIS2"]["p99"].mean()
        o3_p99_mean = ok_visits[ok_visits["site"] == "OASIS3"]["p99"].mean()
        
        p1_diff_pct = 100 * abs(o2_p1_mean - o3_p1_mean) / ((o2_p1_mean + o3_p1_mean) / 2)
        p99_diff_pct = 100 * abs(o2_p99_mean - o3_p99_mean) / ((o2_p99_mean + o3_p99_mean) / 2)
        
        print(f"\n  Site intensity difference after normalization:")
        print(f"    p1:  OASIS2={o2_p1_mean:.1f}, OASIS3={o3_p1_mean:.1f}, diff={p1_diff_pct:.1f}%")
        print(f"    p99: OASIS2={o2_p99_mean:.1f}, OASIS3={o3_p99_mean:.1f}, diff={p99_diff_pct:.1f}%")
        
        if p1_diff_pct < 15 and p99_diff_pct < 15:
            print(f"  ✓ Good multi-site normalization (diff <15%)")
        else:
            print(f"  ⚠️  Site differences may indicate normalization issues")
    
    # Sample preprocessed files
    print("\n--- Sample Preprocessed Files (Verification) ---")
    for i, row in ok_visits.head(3).iterrows():
        print(f"\n  {row['subject_id']} / {row['visit_code']} (Site: {row['site']}, Diagnosis: {int(row['diagnosis_visit'])})")
        print(f"    Raw:  {row['raw_nifti_path']}")
        print(f"    Proc: {row['proc_nifti_path']}")
        
        # Verify file exists and check shape
        if os.path.exists(row['proc_nifti_path']):
            img_check = nib.load(row['proc_nifti_path'])
            data_check = img_check.get_fdata()
            print(f"    ✓ File exists")
            print(f"      Shape: {img_check.shape} (matches MNI: {img_check.shape == mni_shape})")
            print(f"      Range: [{data_check.min():.3f}, {data_check.max():.3f}]")
            print(f"      Brain voxels nonzero: {(data_check > 0).sum():,}")
        else:
            print(f"    ❌ File not found!")

# ============================================================================
# STEP 4: VALIDATION CHECKS
# ============================================================================

print("\n" + "=" * 80)
print("VALIDATION CHECKS")
print("=" * 80)

# Check 1: Success rate
success_rate = n_success / len(visits_proc) if len(visits_proc) > 0 else 0
print(f"\n✓ Overall success rate: {100*success_rate:.1f}%")

if success_rate < 0.95:
    print(f"  ⚠️  WARNING: Success rate below 95% target")
    if n_fail > 0:
        print(f"\n  Failed visits by dataset:")
        failed = visits_preproc[visits_preproc["preproc_ok"] == False]
        fail_breakdown = failed.groupby("dataset").size()
        print(fail_breakdown)
        
        print(f"\n  Sample failure reasons:")
        for reason, count in failed["error_msg"].value_counts().head(5).items():
            print(f"    {count}x: {reason[:100]}")
else:
    print(f"  ✓ Excellent success rate (target: >98%)")

# Check 2: Both datasets represented
if len(ok_visits) > 0:
    datasets_ok = ok_visits["dataset"].unique()
    print(f"\n✓ Datasets in successful preprocessing: {list(datasets_ok)}")
    
    if "OASIS2" not in datasets_ok:
        print(f"  ❌ OASIS2 missing from successful preprocessing")
    elif "OASIS3" not in datasets_ok:
        print(f"  ❌ OASIS3 missing from successful preprocessing")
    else:
        print(f"  ✓ Both OASIS2 and OASIS3 present")
        
        # Dataset-specific success rates
        o2_success = ok_visits[ok_visits["dataset"] == "OASIS2"].shape[0]
        o2_total = visits_proc[visits_proc["dataset"] == "OASIS2"].shape[0]
        o3_success = ok_visits[ok_visits["dataset"] == "OASIS3"].shape[0]
        o3_total = visits_proc[visits_proc["dataset"] == "OASIS3"].shape[0]
        
        print(f"  OASIS2: {o2_success}/{o2_total} ({100*o2_success/o2_total:.1f}%)")
        print(f"  OASIS3: {o3_success}/{o3_total} ({100*o3_success/o3_total:.1f}%)")

# Check 3: Both diagnoses represented
if len(ok_visits) > 0:
    diag_ok = sorted(ok_visits["diagnosis_visit"].unique())
    print(f"\n✓ Diagnoses in successful preprocessing: {diag_ok}")
    
    if 0 not in diag_ok:
        print(f"  ❌ CN (diagnosis=0) missing")
    elif 1 not in diag_ok:
        print(f"  ❌ AD (diagnosis=1) missing")
    else:
        print(f"  ✓ Both CN and AD classes present")
        
        cn_count = (ok_visits["diagnosis_visit"] == 0).sum()
        ad_count = (ok_visits["diagnosis_visit"] == 1).sum()
        print(f"  CN: {cn_count}, AD: {ad_count}, Ratio: {cn_count/ad_count:.2f}:1")

# Check 4: Percentile sanity
if len(ok_visits) > 0:
    extreme_p1 = ok_visits[ok_visits["p1"] < -1000]
    extreme_p99 = ok_visits[ok_visits["p99"] > 10000]
    identical = ok_visits[abs(ok_visits["p99"] - ok_visits["p1"]) < 1.0]
    
    print(f"\n✓ Percentile sanity checks:")
    print(f"  Extreme p1 (<-1000): {len(extreme_p1)} visits")
    print(f"  Extreme p99 (>10000): {len(extreme_p99)} visits")
    print(f"  Near-identical p1≈p99: {len(identical)} visits")
    
    if len(extreme_p1) == 0 and len(extreme_p99) == 0 and len(identical) == 0:
        print(f"  ✓ All percentile values are reasonable")
    else:
        print(f"  ⚠️  Some percentile outliers detected")

# Check 5: Expected visit counts
expected_min = 560  # Target >97% of 575
if len(ok_visits) >= expected_min:
    print(f"\n✓ Preprocessed visits ({len(ok_visits)}) meets target (≥{expected_min})")
else:
    print(f"\n⚠️  WARNING: Only {len(ok_visits)} visits preprocessed (expected ≥{expected_min})")

# ============================================================================
# STEP 5: CLEANUP & SAVE OUTPUT
# ============================================================================

print("\n" + "=" * 80)
print("CLEANUP & SAVE OUTPUT")
print("=" * 80)

# Clean up temporary files
if os.path.exists(TEMP_DIR):
    temp_files = list(Path(TEMP_DIR).glob("*"))
    if temp_files:
        print(f"\nCleaning up {len(temp_files)} temporary cleaned volume files...")
        shutil.rmtree(TEMP_DIR)
        print(f"  ✓ Removed temp directory: {TEMP_DIR}")

# Save output CSV
output_csv = "visits_preproc.csv"
visits_preproc.to_csv(output_csv, index=False)

print(f"\n✓ Saved: {output_csv}")
print(f"  Shape: {visits_preproc.shape}")
print(f"  Columns: {list(visits_preproc.columns)}")

print("\n--- Preview: Successful Visits ---")
preview_cols = ["subject_id", "visit_code", "site", "diagnosis_visit", "p1", "p99", "preproc_ok"]
if len(ok_visits) > 0:
    print(visits_preproc[visits_preproc["preproc_ok"]][preview_cols].head(5))

if n_fail > 0:
    print("\n--- Preview: Failed Visits ---")
    fail_cols = ["subject_id", "visit_code", "dataset", "error_msg"]
    print(visits_preproc[~visits_preproc["preproc_ok"]][fail_cols].head(5))

print("\n" + "=" * 80)
print("SNIPPET S2: COMPLETE ✓")
print("=" * 80)
print("\n📊 Summary:")
print(f"  • Preprocessed: {n_success}/{len(visits_proc)} visits ({100*n_success/len(visits_proc):.1f}%)")
print(f"  • OASIS2: {(ok_visits['dataset']=='OASIS2').sum()} successful")
print(f"  • OASIS3: {(ok_visits['dataset']=='OASIS3').sum()} successful")
print(f"  • Output directory: {OUTPUT_ROOT}/")
print(f"  • Metadata CSV: {output_csv}")
print(f"  • Ready for S3 (tri-planar slice extraction)")
print("=" * 80)


SNIPPET S2: 3D PREPROCESSING PIPELINE (4D HANDLING + OASIS-3 FIXED)

Configuration:
  Output root: processed_mri
  MNI resolution: 1mm isotropic
  Registration DOF: 6 (rigid)
  Max iterations: 200
  Temp directory: temp_cleaned_volumes

[STEP 0] Loading visits table...
  Total visits in table: 796
  Visits to preprocess: 575

--- Breakdown by Dataset & Diagnosis ---
diagnosis_visit  0.0  1.0
dataset                  
OASIS2           206   42
OASIS3           293   34

[STEP 1] Loading MNI template and brain mask via nilearn...
  (First run will download and cache ~50MB, subsequent runs are instant)

  Loading MNI152 T1 template...
  ✓ MNI template loaded
    Shape: (197, 233, 189)
    Voxel size: (1.0, 1.0, 1.0) mm
    Data range: [0.0, 1.0]

  Loading MNI152 brain mask...
  ✓ Brain mask loaded
    Shape: (197, 233, 189)
    Brain voxels: 1,882,989
    Brain fraction: 21.7%

  Saving template to disk for SimpleITK...
    ✓ Saved template: mni_template/mni_icbm152_t1_1mm.nii.gz
    ✓ S

Preprocessing:  45%|████████████████████▊                         | 260/575 [19:42<17:43,  3.38s/it]


⚠️  Preprocessing failed for OASIS3|OAS30026|d0129|1|sub-OAS30026_sess-d0129_T1w.nii
    Dataset: OASIS3, Subject: OAS30026, Visit: d0129
    Raw path: /kaggle/input/oaisis-3/oaisis3/OAS30026_MR_d0129/anat4/NIFTI/sub-OAS30026_ses-d0129_T1w.nii/sub-OAS30026_sess-d0129_T1w.nii
    Error: Volume preparation failed: Nibabel load failed: Expected 21626880 bytes, got 19597910 bytes from /kaggle/input/oaisis-3/oaisis3/OAS30026_MR_d0129/anat4/NIFTI/sub-OAS30026_ses-d0129_T1w.nii/sub-OAS3002


Preprocessing:  45%|████████████████████▉                         | 261/575 [19:42<12:48,  2.45s/it]


⚠️  Preprocessing failed for OASIS3|OAS30027|d0433|1|sub-OAS30027_sess-d0433_run-01_T1w.nii
    Dataset: OASIS3, Subject: OAS30027, Visit: d0433
    Raw path: /kaggle/input/oaisis-3/oaisis3/OAS30027_MR_d0433/anat2/NIFTI/sub-OAS30027_ses-d0433_run-01_T1w.nii/sub-OAS30027_sess-d0433_run-01_T1w.nii
    Error: Volume preparation failed: Nibabel load failed: Expected 23068672 bytes, got 22027468 bytes from /kaggle/input/oaisis-3/oaisis3/OAS30027_MR_d0433/anat2/NIFTI/sub-OAS30027_ses-d0433_run-01_T1w.nii/sub-


...


⚠️  Preprocessing failed for OASIS3|OAS30073|d1587|1|sub-OAS30073_sess-d1587_run-01_T1w.nii
    Dataset: OASIS3, Subject: OAS30073, Visit: d1587
    Raw path: /kaggle/input/oaisis-3/oaisis3/OAS30073_MR_d1587/anat2/NIFTI/sub-OAS30073_ses-d1587_run-01_T1w.nii/sub-OAS30073_sess-d1587_run-01_T1w.nii
    Error: Volume preparation failed: Nibabel load failed: Expected 23068672 bytes, got 22957244 bytes from /kaggle/input/oaisis-3/oaisis3/OAS30073_MR_d1587/anat2/NIFTI/sub-OAS30073_ses-d1587_run-01_T1w.nii/sub-


Preprocessing:  49%|██████████████████████▋                       | 284/575 [21:22<14:59,  3.09s/it]


⚠️  Preprocessing failed for OASIS3|OAS30074|d0049|1|sub-OAS30074_sess-d0049_run-01_T1w.nii
    Dataset: OASIS3, Subject: OAS30074, Visit: d0049
    Raw path: /kaggle/input/oaisis-3/oaisis3/OAS30074_MR_d0049/anat2/NIFTI/sub-OAS30074_ses-d0049_run-01_T1w.nii/sub-OAS30074_sess-d0049_run-01_T1w.nii
    Error: Volume preparation failed: Nibabel load failed: Expected 23068672 bytes, got 22402866 bytes from /kaggle/input/oaisis-3/oaisis3/OAS30074_MR_d0049/anat2/NIFTI/sub-OAS30074_ses-d0049_run-01_T1w.nii/sub-


Preprocessing: 100%|██████████████████████████████████████████████| 575/575 [41:26<00:00,  4.32s/it]



✓ Preprocessing complete
  Total successful: 517/575 (89.9%)
  OASIS2 failures: 0/248
  OASIS3 failures: 58/327

QUALITY CONTROL STATISTICS

Preprocessed visits (OK): 517/575

--- Successful Preprocessing by Dataset & Diagnosis ---
diagnosis_visit  0.0  1.0
dataset                  
OASIS2           206   42
OASIS3           241   28

--- By Site & Diagnosis ---
diagnosis_visit  0.0  1.0
site                     
OASIS2           206   42
OASIS3           241   28

--- Percentile Statistics (p1) ---
count    517.000000
mean      81.807588
std       69.725517
min        6.686989
25%       17.716341
50%       27.580527
75%      149.088284
max      374.049250
Name: p1, dtype: float64

--- Percentile Statistics (p99) ---
count     517.000000
mean     1427.771564
std      1124.693459
min       170.160726
25%       392.267673
50%       527.465894
75%      2594.925098
max      4095.000000
Name: p99, dtype: float64

--- Percentile Stats by Site (Multi-site Normalization Check) ---
           

In [4]:
# ============================================================================
# SNIPPET S3: TRI-PLANAR SLICE GENERATOR (32 SLICES PER VISIT, 160×160)
# ============================================================================
# Extracts centered tri-planar slices from MNI-normalized volumes.
# Outputs: ~16,544 PNG slices + metadata CSV for training pipeline.

import os
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
from tqdm import tqdm
import warnings

from skimage.transform import resize
from PIL import Image

warnings.filterwarnings("ignore")

print("=" * 80)
print("SNIPPET S3: TRI-PLANAR SLICE GENERATOR")
print("=" * 80)

# ============================================================================
# CONFIGURATION
# ============================================================================

SLICE_ROOT = "tri_slices_160"
os.makedirs(SLICE_ROOT, exist_ok=True)

TARGET_SIZE = (160, 160)

# Slice counts per plane
N_AXIAL = 16
N_CORONAL = 8
N_SAGITTAL = 8
TOTAL_SLICES_PER_VISIT = N_AXIAL + N_CORONAL + N_SAGITTAL  # 32

print(f"\nConfiguration:")
print(f"  Slice root: {SLICE_ROOT}")
print(f"  Target size: {TARGET_SIZE[0]}×{TARGET_SIZE[1]} pixels")
print(f"  Slices per visit:")
print(f"    Axial: {N_AXIAL}")
print(f"    Coronal: {N_CORONAL}")
print(f"    Sagittal: {N_SAGITTAL}")
print(f"    Total: {TOTAL_SLICES_PER_VISIT}")

# ============================================================================
# STEP 0: LOAD PREPROCESSED VISITS
# ============================================================================

print("\n[STEP 0] Loading visits_preproc.csv...")

visits_preproc = pd.read_csv("visits_preproc.csv")
print(f"  Total rows in visits_preproc: {len(visits_preproc)}")

visits_ok = visits_preproc[visits_preproc["preproc_ok"] == True].copy()
print(f"  Visits with preproc_ok=True: {len(visits_ok)}")

print("\n--- Breakdown by Dataset & Diagnosis (preprocessed only) ---")
breakdown = visits_ok.groupby(["dataset", "diagnosis_visit"]).size().unstack(fill_value=0)
print(breakdown)

expected_slices = len(visits_ok) * TOTAL_SLICES_PER_VISIT
print(f"\nExpected total slices: ~{expected_slices:,} ({len(visits_ok)} visits × {TOTAL_SLICES_PER_VISIT})")

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def get_indices(center, n_slices, dim_size):
    """
    Get 'n_slices' indices centered around 'center' within [0, dim_size).
    Ensures we always return exactly n_slices valid indices.
    
    Args:
        center: Center index (int)
        n_slices: Number of slices to extract
        dim_size: Maximum dimension size
    
    Returns:
        List of n_slices indices
    """
    center = int(center)
    half = n_slices // 2
    start = center - half
    
    # Ensure start is valid
    if start < 0:
        start = 0
    
    end = start + n_slices
    
    # Ensure end is valid
    if end > dim_size:
        end = dim_size
        start = max(0, end - n_slices)
    
    indices = list(range(start, end))
    
    # Validation
    assert len(indices) == n_slices, f"Expected {n_slices} indices, got {len(indices)}"
    assert all(0 <= i < dim_size for i in indices), "Invalid indices generated"
    
    return indices


def find_brain_center(volume):
    """
    Compute brain center from non-zero voxels (skull-stripped volume).
    
    Args:
        volume: 3D numpy array (X, Y, Z) with values in [0, 1]
    
    Returns:
        Tuple (cx, cy, cz) of center coordinates
    """
    mask = volume > 0
    
    if not np.any(mask):
        raise ValueError("No non-zero voxels found (brain mask empty)")
    
    # Get indices of non-zero voxels
    xs, ys, zs = np.where(mask)
    
    # Compute center as mean position
    cx = int(np.round(xs.mean()))
    cy = int(np.round(ys.mean()))
    cz = int(np.round(zs.mean()))
    
    return cx, cy, cz


def save_slice_png(slice_2d, out_path, target_size=TARGET_SIZE):
    """
    Resize 2D slice to target_size and save as grayscale PNG.
    
    Args:
        slice_2d: 2D numpy array with values in [0, 1]
        out_path: Output path for PNG
        target_size: Target (height, width) tuple
    """
    # Handle NaN/Inf values
    slice_2d = np.nan_to_num(slice_2d, nan=0.0, posinf=1.0, neginf=0.0)
    
    # Ensure [0, 1] range
    slice_2d = np.clip(slice_2d, 0.0, 1.0)
    
    # Resize with anti-aliasing
    slice_resized = resize(
        slice_2d,
        target_size,
        order=1,  # Bilinear interpolation
        mode="constant",
        cval=0.0,
        anti_aliasing=True,
        preserve_range=True,
    )
    
    # Convert to uint8
    slice_uint8 = np.clip(slice_resized * 255.0, 0, 255).astype(np.uint8)
    
    # Save as grayscale PNG
    img = Image.fromarray(slice_uint8, mode="L")
    img.save(out_path)


# ============================================================================
# STEP 1: GENERATE TRI-PLANAR SLICES
# ============================================================================

print("\n[STEP 1] Generating tri-planar slices...")
print("  (This will take ~10-20 minutes for 517 visits)\n")

rows_meta = []
n_visits_success = 0
n_visits_failed = 0
n_slices_total = 0

for idx, row in tqdm(visits_ok.iterrows(), total=len(visits_ok), desc="Processing visits", ncols=100):
    dataset = row["dataset"]
    site = row["site"]
    subject_id = row["subject_id"]
    visit_code = row["visit_code"]
    scan_uid = row["scan_uid"]
    label = row["diagnosis_visit"]
    proc_path = row["proc_nifti_path"]
    
    # Validate path exists
    if not isinstance(proc_path, str) or not os.path.exists(proc_path):
        n_visits_failed += 1
        if n_visits_failed <= 3:
            print(f"\n⚠️  Skipping {scan_uid}: preprocessed file not found")
        continue
    
    try:
        # Load preprocessed MNI-normalized volume
        img = nib.load(proc_path)
        vol = img.get_fdata().astype(np.float32)  # shape (X, Y, Z)
        
        # Validate 3D
        if vol.ndim != 3:
            raise ValueError(f"Expected 3D volume, got shape {vol.shape}")
        
        X, Y, Z = vol.shape
        
        # Find brain center from non-zero voxels
        cx, cy, cz = find_brain_center(vol)
        
        # Generate slice indices centered around brain
        axial_idxs = get_indices(cz, N_AXIAL, Z)      # Along Z axis
        coronal_idxs = get_indices(cy, N_CORONAL, Y)  # Along Y axis
        sagittal_idxs = get_indices(cx, N_SAGITTAL, X) # Along X axis
        
        # Create output directory for this visit
        out_dir = os.path.join(SLICE_ROOT, dataset, subject_id, str(visit_code))
        os.makedirs(out_dir, exist_ok=True)
        
        # -----------------------------------------------
        # AXIAL SLICES (16 slices along Z axis)
        # -----------------------------------------------
        for k, z in enumerate(axial_idxs):
            slice_2d = vol[:, :, z]  # Shape: (X, Y)
            plane = "axial"
            slice_idx = k
            
            filename = f"{subject_id}_{visit_code}_{plane}_s{slice_idx:02d}.png"
            img_path = os.path.join(out_dir, filename)
            
            save_slice_png(slice_2d, img_path)
            
            rows_meta.append({
                "dataset": dataset,
                "site": site,
                "subject_id": subject_id,
                "visit_code": visit_code,
                "scan_uid": scan_uid,
                "diagnosis_binary": int(label),
                "plane": plane,
                "slice_idx": slice_idx,
                "axis_index": int(z),
                "x_center": int(cx),
                "y_center": int(cy),
                "z_center": int(cz),
                "img_path": img_path,
            })
            n_slices_total += 1
        
        # -----------------------------------------------
        # CORONAL SLICES (8 slices along Y axis)
        # -----------------------------------------------
        for k, y in enumerate(coronal_idxs):
            slice_2d = vol[:, y, :]  # Shape: (X, Z)
            plane = "coronal"
            slice_idx = k
            
            filename = f"{subject_id}_{visit_code}_{plane}_s{slice_idx:02d}.png"
            img_path = os.path.join(out_dir, filename)
            
            save_slice_png(slice_2d, img_path)
            
            rows_meta.append({
                "dataset": dataset,
                "site": site,
                "subject_id": subject_id,
                "visit_code": visit_code,
                "scan_uid": scan_uid,
                "diagnosis_binary": int(label),
                "plane": plane,
                "slice_idx": slice_idx,
                "axis_index": int(y),
                "x_center": int(cx),
                "y_center": int(cy),
                "z_center": int(cz),
                "img_path": img_path,
            })
            n_slices_total += 1
        
        # -----------------------------------------------
        # SAGITTAL SLICES (8 slices along X axis)
        # -----------------------------------------------
        for k, x in enumerate(sagittal_idxs):
            slice_2d = vol[x, :, :]  # Shape: (Y, Z)
            plane = "sagittal"
            slice_idx = k
            
            filename = f"{subject_id}_{visit_code}_{plane}_s{slice_idx:02d}.png"
            img_path = os.path.join(out_dir, filename)
            
            save_slice_png(slice_2d, img_path)
            
            rows_meta.append({
                "dataset": dataset,
                "site": site,
                "subject_id": subject_id,
                "visit_code": visit_code,
                "scan_uid": scan_uid,
                "diagnosis_binary": int(label),
                "plane": plane,
                "slice_idx": slice_idx,
                "axis_index": int(x),
                "x_center": int(cx),
                "y_center": int(cy),
                "z_center": int(cz),
                "img_path": img_path,
            })
            n_slices_total += 1
        
        n_visits_success += 1
        
    except Exception as e:
        n_visits_failed += 1
        if n_visits_failed <= 5:
            print(f"\n⚠️  Slice generation failed for {scan_uid}")
            print(f"    Dataset: {dataset}, Subject: {subject_id}, Visit: {visit_code}")
            print(f"    Error: {str(e)[:150]}")
        continue

print(f"\n✓ Slice generation complete")
print(f"  Visits processed successfully: {n_visits_success}/{len(visits_ok)}")
print(f"  Visits failed: {n_visits_failed}")
print(f"  Total slices generated: {n_slices_total:,}")
print(f"  Expected: {expected_slices:,}")
print(f"  Average slices per visit: {n_slices_total/n_visits_success:.1f}" if n_visits_success > 0 else "")

# ============================================================================
# STEP 2: SAVE METADATA & QC
# ============================================================================

print("\n[STEP 2] Saving metadata CSV and QC validation...")

slices_df = pd.DataFrame(rows_meta)
output_csv = "slice_level_tri_views_160.csv"
slices_df.to_csv(output_csv, index=False)

print(f"\n✓ Saved: {output_csv}")
print(f"  Shape: {slices_df.shape}")
print(f"  Columns: {list(slices_df.columns)}")

# ============================================================================
# QC STATISTICS
# ============================================================================

print("\n" + "=" * 80)
print("QUALITY CONTROL STATISTICS")
print("=" * 80)

# Plane distribution
print("\n--- Slice Distribution by Plane ---")
plane_counts = slices_df["plane"].value_counts().sort_index()
print(plane_counts)
print(f"\nExpected: Axial={N_AXIAL*n_visits_success}, Coronal={N_CORONAL*n_visits_success}, Sagittal={N_SAGITTAL*n_visits_success}")

# Diagnosis distribution
print("\n--- Slice Distribution by Diagnosis ---")
diag_counts = slices_df["diagnosis_binary"].value_counts().sort_index()
print(diag_counts)
if 0 in diag_counts.index and 1 in diag_counts.index:
    cn_slices = diag_counts[0]
    ad_slices = diag_counts[1]
    ratio = cn_slices / ad_slices if ad_slices > 0 else float('inf')
    print(f"\nClass ratio: CN:AD = {ratio:.2f}:1")
    print(f"  CN slices: {cn_slices:,}")
    print(f"  AD slices: {ad_slices:,}")

# Dataset distribution
print("\n--- Slice Distribution by Dataset ---")
dataset_counts = slices_df["dataset"].value_counts()
print(dataset_counts)

# Per-visit slice count validation
print("\n--- Per-Visit Slice Count Validation ---")
visit_slice_counts = slices_df.groupby("scan_uid").size()
print(f"  Mean slices per visit: {visit_slice_counts.mean():.1f}")
print(f"  Min slices per visit: {visit_slice_counts.min()}")
print(f"  Max slices per visit: {visit_slice_counts.max()}")

if visit_slice_counts.min() != TOTAL_SLICES_PER_VISIT or visit_slice_counts.max() != TOTAL_SLICES_PER_VISIT:
    print(f"  ⚠️  WARNING: Some visits don't have exactly {TOTAL_SLICES_PER_VISIT} slices")
    irregular = visit_slice_counts[visit_slice_counts != TOTAL_SLICES_PER_VISIT]
    print(f"  Irregular visits: {len(irregular)}")
else:
    print(f"  ✓ All visits have exactly {TOTAL_SLICES_PER_VISIT} slices")

# Sample rows
print("\n--- Sample Metadata Rows ---")
print(slices_df.head(10))

# File existence check
print("\n--- File Existence Validation ---")
sample_paths = slices_df["img_path"].sample(min(5, len(slices_df))).tolist()
all_exist = True
for path in sample_paths:
    exists = os.path.exists(path)
    print(f"  {exists} : {path}")
    if not exists:
        all_exist = False

if all_exist:
    print(f"\n✓ All sampled slice files exist on disk")
else:
    print(f"\n⚠️  Some slice files missing")

# ============================================================================
# VALIDATION CHECKS
# ============================================================================

print("\n" + "=" * 80)
print("VALIDATION CHECKS")
print("=" * 80)

# Check 1: Total slice count
expected_min = int(0.95 * expected_slices)  # Allow 5% tolerance
if n_slices_total >= expected_min:
    print(f"\n✓ Total slices ({n_slices_total:,}) meets target (≥{expected_min:,})")
else:
    print(f"\n⚠️  WARNING: Only {n_slices_total:,} slices generated (expected ≥{expected_min:,})")

# Check 2: Both diagnoses present
if 0 in diag_counts.index and 1 in diag_counts.index:
    print(f"✓ Both CN and AD classes present in slices")
else:
    print(f"❌ Missing diagnosis class in slices")

# Check 3: All planes present
if set(plane_counts.index) == {"axial", "coronal", "sagittal"}:
    print(f"✓ All three planes (axial, coronal, sagittal) present")
else:
    print(f"⚠️  Not all planes present: {list(plane_counts.index)}")

# Check 4: Both datasets present
if "OASIS2" in dataset_counts.index and "OASIS3" in dataset_counts.index:
    print(f"✓ Both OASIS2 and OASIS3 datasets present")
else:
    print(f"⚠️  Missing dataset: {list(dataset_counts.index)}")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("SNIPPET S3: COMPLETE ✓")
print("=" * 80)

print(f"\n📊 Summary:")
print(f"  • Visits processed: {n_visits_success}/{len(visits_ok)}")
print(f"  • Total slices: {n_slices_total:,}")
print(f"  • Axial slices: {plane_counts.get('axial', 0):,}")
print(f"  • Coronal slices: {plane_counts.get('coronal', 0):,}")
print(f"  • Sagittal slices: {plane_counts.get('sagittal', 0):,}")
print(f"  • Output directory: {SLICE_ROOT}/")
print(f"  • Metadata CSV: {output_csv}")
print(f"  • Class balance: CN {cn_slices:,} vs AD {ad_slices:,} ({ratio:.2f}:1)" if 'ratio' in locals() else "")
print(f"\n  Ready for S4 (subject-level splits)")
print("=" * 80)

SNIPPET S3: TRI-PLANAR SLICE GENERATOR

Configuration:
  Slice root: tri_slices_160
  Target size: 160×160 pixels
  Slices per visit:
    Axial: 16
    Coronal: 8
    Sagittal: 8
    Total: 32

[STEP 0] Loading visits_preproc.csv...
  Total rows in visits_preproc: 575
  Visits with preproc_ok=True: 517

--- Breakdown by Dataset & Diagnosis (preprocessed only) ---
diagnosis_visit  0.0  1.0
dataset                  
OASIS2           206   42
OASIS3           241   28

Expected total slices: ~16,544 (517 visits × 32)

[STEP 1] Generating tri-planar slices...
  (This will take ~10-20 minutes for 517 visits)



Processing visits: 100%|██████████████████████████████████████████| 517/517 [03:23<00:00,  2.54it/s]


✓ Slice generation complete
  Visits processed successfully: 517/517
  Visits failed: 0
  Total slices generated: 16,544
  Expected: 16,544
  Average slices per visit: 32.0

[STEP 2] Saving metadata CSV and QC validation...

✓ Saved: slice_level_tri_views_160.csv
  Shape: (16544, 13)
  Columns: ['dataset', 'site', 'subject_id', 'visit_code', 'scan_uid', 'diagnosis_binary', 'plane', 'slice_idx', 'axis_index', 'x_center', 'y_center', 'z_center', 'img_path']

QUALITY CONTROL STATISTICS

--- Slice Distribution by Plane ---
plane
axial       8272
coronal     4136
sagittal    4136
Name: count, dtype: int64

Expected: Axial=8272, Coronal=4136, Sagittal=4136

--- Slice Distribution by Diagnosis ---
diagnosis_binary
0    14304
1     2240
Name: count, dtype: int64

Class ratio: CN:AD = 6.39:1
  CN slices: 14,304
  AD slices: 2,240

--- Slice Distribution by Dataset ---
dataset
OASIS3    8608
OASIS2    7936
Name: count, dtype: int64

--- Per-Visit Slice Count Validation ---
  Mean slices per vis

In [5]:
# ============================================================================
# SNIPPET S4: SUBJECT-LEVEL SPLITS & LOCKED TEST SET
# ============================================================================
# Creates stratified subject-level train/val/test splits with 5-fold CV.
# Locks 20% test set (never to be touched during development).
# Propagates splits to visit and slice levels.

import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import StratifiedKFold, train_test_split
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("SNIPPET S4: SUBJECT-LEVEL SPLITS & LOCKED TEST SET")
print("=" * 80)

# ============================================================================
# CONFIGURATION
# ============================================================================

RANDOM_SEED = 42
TEST_SIZE = 0.20  # 20% subjects for locked test set
N_FOLDS = 5

# Date stamp for locked test set
DATE_STAMP = datetime.now().strftime("%Y%m%d")

print(f"\nConfiguration:")
print(f"  Random seed: {RANDOM_SEED}")
print(f"  Test set size: {TEST_SIZE:.0%} of subjects")
print(f"  CV folds: {N_FOLDS}")
print(f"  Date stamp: {DATE_STAMP}")

# ============================================================================
# STEP 1: BUILD SUBJECT-LEVEL TABLE
# ============================================================================

print("\n[STEP 1] Building subject-level table from preprocessed visits...")

# Load visits from S2
visits_preproc = pd.read_csv("visits_preproc.csv")
print(f"  Loaded visits_preproc.csv: {len(visits_preproc)} rows")

# Filter to visits that are classifier-eligible AND successfully preprocessed
visits_model = visits_preproc[
    (visits_preproc["include_for_classifier"] == True) & 
    (visits_preproc["preproc_ok"] == True)
].copy()

print(f"  Visits for modeling (eligible + preprocessed): {len(visits_model)}")

# Group by subject to build subject-level statistics
subject_rows = []

for subject_id, grp in visits_model.groupby("subject_id"):
    # Dataset and site should be consistent per subject
    dataset = grp["dataset"].iloc[0]
    site = grp["site"].iloc[0]
    
    n_visits_total = len(grp)
    n_AD_visits = (grp["diagnosis_visit"] == 1).sum()
    n_CN_visits = (grp["diagnosis_visit"] == 0).sum()
    
    # Subject diagnosis: has_AD = 1 if any visit is AD
    has_AD = 1 if n_AD_visits > 0 else 0
    
    subject_rows.append({
        "subject_id": subject_id,
        "dataset": dataset,
        "site": site,
        "n_visits_total": n_visits_total,
        "n_CN_visits": n_CN_visits,
        "n_AD_visits": n_AD_visits,
        "has_AD": has_AD,
    })

subjects_df = pd.DataFrame(subject_rows)

print(f"\n✓ Subject-level table created: {len(subjects_df)} subjects")

print("\n--- Subject Distribution by Dataset & Diagnosis ---")
subject_breakdown = subjects_df.groupby(["dataset", "has_AD"]).size().unstack(fill_value=0)
subject_breakdown.columns = ["CN (has_AD=0)", "AD (has_AD=1)"]
print(subject_breakdown)

# Validation: ensure both datasets have AD subjects
for ds in ["OASIS2", "OASIS3"]:
    if ds in subjects_df["dataset"].values:
        n_ad_in_ds = subjects_df[(subjects_df["dataset"] == ds) & (subjects_df["has_AD"] == 1)].shape[0]
        if n_ad_in_ds == 0:
            print(f"\n⚠️  WARNING: No AD subjects in {ds}")
        else:
            print(f"  ✓ {ds}: {n_ad_in_ds} AD subjects")

# ============================================================================
# STEP 2: STRATIFIED SUBJECT-LEVEL TRAIN/TEST SPLIT
# ============================================================================

print("\n[STEP 2] Creating stratified 80/20 train-val/test split...")

# Create stratification label: combine dataset and diagnosis
subjects_df["stratum"] = subjects_df["dataset"] + "_" + subjects_df["has_AD"].astype(str)

print(f"\nStratification groups:")
print(subjects_df["stratum"].value_counts().sort_index())

# Perform stratified split
train_val_indices, test_indices = train_test_split(
    np.arange(len(subjects_df)),
    test_size=TEST_SIZE,
    random_state=RANDOM_SEED,
    stratify=subjects_df["stratum"]
)

# Assign split labels
subjects_df["subject_split"] = "trainval"
subjects_df.loc[test_indices, "subject_split"] = "test"

# Initialize cv_fold (will be set properly in next step)
subjects_df["cv_fold"] = -1

print(f"\n✓ Split created:")
print(f"  Train+Val subjects: {len(train_val_indices)} ({100*len(train_val_indices)/len(subjects_df):.1f}%)")
print(f"  Test subjects: {len(test_indices)} ({100*len(test_indices)/len(subjects_df):.1f}%)")

# Validation: check stratification worked
print("\n--- Split Distribution by Dataset & Diagnosis ---")
split_breakdown = subjects_df.groupby(["dataset", "has_AD", "subject_split"]).size().unstack(fill_value=0)
print(split_breakdown)

# Check for AD subjects in test set
test_subjects = subjects_df[subjects_df["subject_split"] == "test"]
n_ad_test = (test_subjects["has_AD"] == 1).sum()
print(f"\n  AD subjects in test set: {n_ad_test}")

if n_ad_test == 0:
    print(f"  ❌ CRITICAL: No AD subjects in test set! Change random seed.")
    raise ValueError("No AD subjects in test set - stratification failed")
else:
    print(f"  ✓ Test set contains AD subjects")

# ============================================================================
# STEP 3: 5-FOLD CV ON TRAIN+VAL SUBJECTS
# ============================================================================

print(f"\n[STEP 3] Creating {N_FOLDS}-fold cross-validation on train+val subjects...")

# Extract train+val subjects
trainval_subjects = subjects_df[subjects_df["subject_split"] == "trainval"].copy()
trainval_subjects = trainval_subjects.reset_index(drop=True)

print(f"  Train+val subjects: {len(trainval_subjects)}")

# Stratified K-Fold
skf = StratifiedKFold(
    n_splits=N_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED + 1  # Different seed from test split
)

# Generate fold assignments
fold_assignments = np.zeros(len(trainval_subjects), dtype=int)

for fold_idx, (train_idx, val_idx) in enumerate(skf.split(trainval_subjects, trainval_subjects["stratum"])):
    fold_assignments[val_idx] = fold_idx

# Assign fold IDs back to trainval subjects
trainval_subjects["cv_fold"] = fold_assignments

# Update main subjects_df with CV folds
subjects_df.loc[subjects_df["subject_split"] == "trainval", "cv_fold"] = trainval_subjects["cv_fold"].values

print(f"\n✓ Cross-validation folds assigned")

# Validation: check fold distribution
print("\n--- CV Fold Distribution ---")
fold_counts = trainval_subjects["cv_fold"].value_counts().sort_index()
print(fold_counts)

print("\n--- Fold Distribution by Dataset & Diagnosis ---")
for fold in range(N_FOLDS):
    fold_data = trainval_subjects[trainval_subjects["cv_fold"] == fold]
    print(f"\nFold {fold}:")
    print(fold_data.groupby(["dataset", "has_AD"]).size())

# Check each fold has both datasets and both diagnoses
for fold in range(N_FOLDS):
    fold_data = trainval_subjects[trainval_subjects["cv_fold"] == fold]
    
    datasets_in_fold = fold_data["dataset"].unique()
    diagnoses_in_fold = fold_data["has_AD"].unique()
    
    if len(datasets_in_fold) < 2:
        print(f"  ⚠️  WARNING: Fold {fold} missing a dataset")
    if len(diagnoses_in_fold) < 2:
        print(f"  ⚠️  WARNING: Fold {fold} missing a diagnosis class")

# ============================================================================
# STEP 4: SAVE SUBJECT-LEVEL SPLITS
# ============================================================================

print("\n[STEP 4] Saving subject-level split files...")

# Full subject splits
subjects_output = "subjects_splits_LOCKED.csv"
subjects_df.to_csv(subjects_output, index=False)
print(f"\n✓ Saved: {subjects_output}")
print(f"  Shape: {subjects_df.shape}")

# Locked test subjects (separate file with date stamp)
test_subjects_output = f"test_subjects_LOCKED_{DATE_STAMP}.csv"
test_subjects_df = subjects_df[subjects_df["subject_split"] == "test"].copy()
test_subjects_df.to_csv(test_subjects_output, index=False)
print(f"\n✓ Saved: {test_subjects_output}")
print(f"  Shape: {test_subjects_df.shape}")
print(f"  TEST SET IS NOW LOCKED - DO NOT REGENERATE")

# Validation: ensure no overlap
trainval_subject_ids = set(subjects_df[subjects_df["subject_split"] == "trainval"]["subject_id"])
test_subject_ids = set(subjects_df[subjects_df["subject_split"] == "test"]["subject_id"])

overlap = trainval_subject_ids & test_subject_ids
if len(overlap) > 0:
    print(f"\n❌ CRITICAL: {len(overlap)} subjects appear in both train+val and test!")
    raise ValueError("Subject overlap between splits detected")
else:
    print(f"\n✓ No subject overlap between train+val and test")

# ============================================================================
# STEP 5: PROPAGATE SPLITS TO VISIT LEVEL
# ============================================================================

print("\n[STEP 5] Propagating splits to visit level...")

# Merge subject splits into visits
visits_with_splits = visits_preproc.merge(
    subjects_df[["subject_id", "subject_split", "cv_fold", "has_AD"]],
    on="subject_id",
    how="left"
)

# Rename columns for clarity
visits_with_splits = visits_with_splits.rename(columns={
    "subject_split": "visit_split",
    "cv_fold": "visit_cv_fold"
})

# Create use_for_model flag
visits_with_splits["use_for_model"] = (
    (visits_with_splits["include_for_classifier"] == True) &
    (visits_with_splits["preproc_ok"] == True)
)

# Save visits with splits
visits_output = "visits_with_splits.csv"
visits_with_splits.to_csv(visits_output, index=False)

print(f"\n✓ Saved: {visits_output}")
print(f"  Shape: {visits_with_splits.shape}")

# QC: verify all visits from same subject have same split
print("\n--- Visit-Level QC ---")
for subject_id, grp in visits_with_splits.groupby("subject_id"):
    unique_splits = grp["visit_split"].nunique()
    unique_folds = grp["visit_cv_fold"].nunique()
    
    if unique_splits > 1 or unique_folds > 1:
        print(f"  ❌ ERROR: Subject {subject_id} has visits in multiple splits/folds")
        raise ValueError("Subject visits have inconsistent splits")

print(f"  ✓ All visits from same subject have consistent splits")

# Visit distribution
print("\n--- Visit Distribution by Split ---")
model_visits = visits_with_splits[visits_with_splits["use_for_model"] == True]
visit_split_counts = model_visits.groupby(["visit_split", "diagnosis_visit"]).size().unstack(fill_value=0)
visit_split_counts.columns = ["CN", "AD"]
print(visit_split_counts)

# ============================================================================
# STEP 6: PROPAGATE SPLITS TO SLICE LEVEL
# ============================================================================

print("\n[STEP 6] Propagating splits to slice level...")

# Load slices from S3
slices_df = pd.read_csv("slice_level_tri_views_160.csv")
print(f"  Loaded slice_level_tri_views_160.csv: {len(slices_df)} rows")

# Build mapping from scan_uid to split/fold
split_map = visits_with_splits.set_index("scan_uid")["visit_split"].to_dict()
fold_map = visits_with_splits.set_index("scan_uid")["visit_cv_fold"].to_dict()

# Map to slices
slices_df["slice_split"] = slices_df["scan_uid"].map(split_map)
slices_df["slice_cv_fold"] = slices_df["scan_uid"].map(fold_map)

# Handle any unmapped slices (shouldn't happen if S2/S3 consistent)
unmapped = slices_df["slice_split"].isna().sum()
if unmapped > 0:
    print(f"\n⚠️  WARNING: {unmapped} slices could not be mapped to splits")
    print(f"  Removing unmapped slices...")
    slices_df = slices_df.dropna(subset=["slice_split"])

# Save slices with splits
slices_output = "slices_with_splits.csv"
slices_df.to_csv(slices_output, index=False)

print(f"\n✓ Saved: {slices_output}")
print(f"  Shape: {slices_df.shape}")

# QC: verify all slices from same visit have same split
print("\n--- Slice-Level QC ---")
for scan_uid, grp in slices_df.groupby("scan_uid"):
    unique_splits = grp["slice_split"].nunique()
    unique_folds = grp["slice_cv_fold"].nunique()
    
    if unique_splits > 1 or unique_folds > 1:
        print(f"  ❌ ERROR: Visit {scan_uid} has slices in multiple splits/folds")
        raise ValueError("Visit slices have inconsistent splits")

print(f"  ✓ All slices from same visit have consistent splits")

# Slice distribution
print("\n--- Slice Distribution by Split & Plane ---")
slice_split_counts = slices_df.groupby(["slice_split", "plane"]).size().unstack(fill_value=0)
print(slice_split_counts)

print("\n--- Slice Distribution by Split & Diagnosis ---")
slice_diag_counts = slices_df.groupby(["slice_split", "diagnosis_binary"]).size().unstack(fill_value=0)
slice_diag_counts.columns = ["CN", "AD"]
print(slice_diag_counts)

# ============================================================================
# STEP 7: COMPREHENSIVE VALIDATION
# ============================================================================

print("\n" + "=" * 80)
print("VALIDATION CHECKS")
print("=" * 80)

# Check 1: Test set size
test_pct = 100 * len(test_subjects_df) / len(subjects_df)
print(f"\n✓ Test set percentage: {test_pct:.1f}% (target: {100*TEST_SIZE:.0f}%)")

if abs(test_pct - 100*TEST_SIZE) > 5:
    print(f"  ⚠️  WARNING: Test set size deviates >5% from target")

# Check 2: Both datasets in test
test_datasets = test_subjects_df["dataset"].unique()
print(f"\n✓ Datasets in test set: {list(test_datasets)}")

if "OASIS2" not in test_datasets or "OASIS3" not in test_datasets:
    print(f"  ⚠️  WARNING: Missing a dataset in test set")

# Check 3: Both diagnoses in test
test_diagnoses = test_subjects_df["has_AD"].unique()
print(f"✓ Diagnoses in test set: {list(test_diagnoses)}")

if 0 not in test_diagnoses or 1 not in test_diagnoses:
    print(f"  ❌ CRITICAL: Missing a diagnosis in test set")

# Check 4: CV fold balance
print(f"\n✓ CV fold sizes:")
for fold in range(N_FOLDS):
    fold_size = (trainval_subjects["cv_fold"] == fold).sum()
    fold_pct = 100 * fold_size / len(trainval_subjects)
    print(f"  Fold {fold}: {fold_size} subjects ({fold_pct:.1f}%)")

# Check 5: Visit counts
test_visit_count = model_visits[model_visits["visit_split"] == "test"].shape[0]
total_visit_count = model_visits.shape[0]
test_visit_pct = 100 * test_visit_count / total_visit_count

print(f"\n✓ Visit-level test percentage: {test_visit_pct:.1f}%")
print(f"  (May differ from subject % due to varying visits per subject)")

# Check 6: Slice counts
test_slice_count = slices_df[slices_df["slice_split"] == "test"].shape[0]
total_slice_count = slices_df.shape[0]
test_slice_pct = 100 * test_slice_count / total_slice_count

print(f"\n✓ Slice-level test percentage: {test_slice_pct:.1f}%")

# Check 7: Plane distribution in test
print(f"\n✓ Plane distribution in test set:")
test_planes = slices_df[slices_df["slice_split"] == "test"]["plane"].value_counts().sort_index()
print(test_planes)

expected_axial = test_visit_count * 16
expected_coronal = test_visit_count * 8
expected_sagittal = test_visit_count * 8

print(f"\n  Expected (based on {test_visit_count} test visits):")
print(f"    Axial: ~{expected_axial}")
print(f"    Coronal: ~{expected_coronal}")
print(f"    Sagittal: ~{expected_sagittal}")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("SNIPPET S4: COMPLETE ✓")
print("=" * 80)

print(f"\n📊 Summary:")
print(f"\n  SUBJECTS:")
print(f"    Total: {len(subjects_df)}")
print(f"    Train+Val: {len(trainval_subjects)} ({100*len(trainval_subjects)/len(subjects_df):.1f}%)")
print(f"    Test (LOCKED): {len(test_subjects_df)} ({100*len(test_subjects_df)/len(subjects_df):.1f}%)")
print(f"    CN subjects: {(subjects_df['has_AD']==0).sum()}")
print(f"    AD subjects: {(subjects_df['has_AD']==1).sum()}")

print(f"\n  VISITS (use_for_model=True):")
print(f"    Total: {len(model_visits)}")
print(f"    Train+Val: {(model_visits['visit_split']=='trainval').sum()}")
print(f"    Test: {(model_visits['visit_split']=='test').sum()}")

print(f"\n  SLICES:")
print(f"    Total: {len(slices_df):,}")
print(f"    Train+Val: {(slices_df['slice_split']=='trainval').sum():,}")
print(f"    Test: {(slices_df['slice_split']=='test').sum():,}")

print(f"\n  OUTPUT FILES:")
print(f"    • {subjects_output}")
print(f"    • {test_subjects_output} ⚠️  LOCKED")
print(f"    • {visits_output}")
print(f"    • {slices_output}")

print(f"\n  Ready for S5 (PyTorch Dataset + DataLoaders)")
print("=" * 80)


SNIPPET S4: SUBJECT-LEVEL SPLITS & LOCKED TEST SET

Configuration:
  Random seed: 42
  Test set size: 20% of subjects
  CV folds: 5
  Date stamp: 20251125

[STEP 1] Building subject-level table from preprocessed visits...
  Loaded visits_preproc.csv: 575 rows
  Visits for modeling (eligible + preprocessed): 517

✓ Subject-level table created: 318 subjects

--- Subject Distribution by Dataset & Diagnosis ---
         CN (has_AD=0)  AD (has_AD=1)
dataset                              
OASIS2              86             25
OASIS3             179             28
  ✓ OASIS2: 25 AD subjects
  ✓ OASIS3: 28 AD subjects

[STEP 2] Creating stratified 80/20 train-val/test split...

Stratification groups:
stratum
OASIS2_0     86
OASIS2_1     25
OASIS3_0    179
OASIS3_1     28
Name: count, dtype: int64

✓ Split created:
  Train+Val subjects: 254 (79.9%)
  Test subjects: 64 (20.1%)

--- Split Distribution by Dataset & Diagnosis ---
subject_split   test  trainval
dataset has_AD                
OASIS2  

In [6]:
# ============================================================================
# SNIPPET S5: TRI-PLANAR VISIT DATASET & DATALOADERS (ENHANCED AUGMENTATION)
# ============================================================================

import os
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

print("=" * 80)
print("SNIPPET S5: TRI-PLANAR VISIT DATASET & DATALOADERS (ENHANCED)")
print("=" * 80)

# ============================================================================
# CONFIGURATION
# ============================================================================

BATCH_SIZE = 8
NUM_WORKERS = 4
TARGET_SIZE = (160, 160)

PLANE_ORDER = ["axial", "coronal", "sagittal"]
N_SLICES_PER_PLANE = {"axial": 16, "coronal": 8, "sagittal": 8}
TOTAL_SLICES = 32

print(f"\nConfiguration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Num workers: {NUM_WORKERS}")
print(f"  Target size: {TARGET_SIZE}")
print(f"  Slices per visit: {TOTAL_SLICES}")
print(f"  Plane order: {PLANE_ORDER}")

# ============================================================================
# CUSTOM COLLATE FUNCTION
# ============================================================================

def custom_collate_fn(batch):
    """Custom collate that keeps metadata as list of dicts."""
    images = torch.stack([item[0] for item in batch])
    labels = torch.stack([item[1] for item in batch])
    metas = [item[2] for item in batch]
    return images, labels, metas

# ============================================================================
# ENHANCED VISIT-LEVEL TRANSFORM
# ============================================================================

class VisitTransform:
    """
    Enhanced augmentation transform applied consistently across all 32 slices.
    
    Geometric transforms (flip, rotation, affine) are applied with SAME parameters
    to all slices to maintain spatial coherence.
    
    Intensity transforms (brightness, contrast, noise) are applied visit-wide.
    """
    
    def __init__(self, is_train=True):
        self.is_train = is_train
        
        if is_train:
            # Geometric augmentation options
            self.geometric_ops = [
                ('hflip', 0.5),
                ('rotation', 10),
                ('affine', (0.05, 0.05)),
            ]
            
            # Intensity augmentation ranges
            self.brightness_range = 0.1  # ±10%
            self.contrast_range = 0.1    # ±10%
            self.noise_std = 0.02        # σ = 0.02 in [0,1] scale
    
    def apply_geometric_aug(self, slices):
        """
        Apply ONE geometric transform to all slices consistently.
        
        Args:
            slices: List of PIL Images (32 slices)
        
        Returns:
            List of augmented PIL Images
        """
        # Randomly select ONE geometric operation
        import random
        op_choice = random.random()
        
        augmented = []
        
        if op_choice < 0.33:  # Horizontal flip
            if random.random() < 0.5:
                augmented = [TF.hflip(img) for img in slices]
            else:
                augmented = slices
        
        elif op_choice < 0.66:  # Rotation
            angle = random.uniform(-10, 10)
            augmented = [TF.rotate(img, angle) for img in slices]
        
        else:  # Affine translation
            translate_x = random.uniform(-0.05, 0.05)
            translate_y = random.uniform(-0.05, 0.05)
            
            # Convert to pixel values
            w, h = slices[0].size
            translate_px = (int(translate_x * w), int(translate_y * h))
            
            augmented = [TF.affine(img, angle=0, translate=translate_px, 
                                   scale=1.0, shear=0) for img in slices]
        
        return augmented
    
    def apply_intensity_aug(self, tensor):
        """
        Apply intensity jitter and Gaussian noise to entire visit.
        
        Args:
            tensor: [K, H, W] tensor in [0, 1]
        
        Returns:
            Augmented tensor [K, H, W]
        """
        # Brightness adjustment (visit-wide)
        brightness_factor = 1.0 + self.brightness_range * (2 * torch.rand(1).item() - 1)
        tensor = tensor * brightness_factor
        
        # Contrast adjustment (visit-wide)
        contrast_factor = 1.0 + self.contrast_range * (2 * torch.rand(1).item() - 1)
        tensor = (tensor - 0.5) * contrast_factor + 0.5
        
        # Clamp after brightness/contrast
        tensor = torch.clamp(tensor, 0.0, 1.0)
        
        # Additive Gaussian noise
        noise = self.noise_std * torch.randn_like(tensor)
        tensor = tensor + noise
        
        # Final clamp
        tensor = torch.clamp(tensor, 0.0, 1.0)
        
        return tensor
    
    def __call__(self, images_list):
        """
        Args:
            images_list: List of 32 PIL Images (grayscale)
        
        Returns:
            tensor: [32, H, W] augmented tensor
        """
        if self.is_train:
            # Apply geometric augmentation (same for all slices)
            images_list = self.apply_geometric_aug(images_list)
        
        # Convert to tensor
        tensor_slices = []
        for img in images_list:
            img_tensor = TF.to_tensor(img)[0]  # Remove channel dimension [1,H,W] → [H,W]
            tensor_slices.append(img_tensor)
        
        tensor = torch.stack(tensor_slices, dim=0)  # [32, H, W]
        
        if self.is_train:
            # Apply intensity augmentation
            tensor = self.apply_intensity_aug(tensor)
        
        return tensor

# ============================================================================
# STEP 1: LOAD SPLITS AND BUILD VISIT-LEVEL INDEX
# ============================================================================

print("\n[STEP 1] Building visit-level index...")

visits_df = pd.read_csv("visits_with_splits.csv")
print(f"  Loaded visits_with_splits.csv: {len(visits_df)} rows")

model_visits = visits_df[visits_df["use_for_model"] == True].copy()
print(f"  Visits usable for modeling: {len(model_visits)}")

slices_df = pd.read_csv("slices_with_splits.csv")
print(f"  Loaded slices_with_splits.csv: {len(slices_df)} rows")

# ============================================================================
# STEP 2: BUILD VISIT RECORDS WITH CANONICAL SLICE ORDERING
# ============================================================================

print("\n[STEP 2] Building visit records with canonical slice ordering...")

visit_records = []
n_valid = 0
n_invalid = 0

for idx, visit_row in model_visits.iterrows():
    scan_uid = visit_row["scan_uid"]
    
    visit_slices = slices_df[slices_df["scan_uid"] == scan_uid].copy()
    
    if len(visit_slices) != TOTAL_SLICES:
        n_invalid += 1
        if n_invalid <= 3:
            print(f"  ⚠️  Skipping {scan_uid}: expected {TOTAL_SLICES} slices, got {len(visit_slices)}")
        continue
    
    plane_counts = visit_slices["plane"].value_counts().to_dict()
    if plane_counts != N_SLICES_PER_PLANE:
        n_invalid += 1
        if n_invalid <= 3:
            print(f"  ⚠️  Skipping {scan_uid}: plane distribution mismatch")
        continue
    
    visit_slices["plane_order"] = visit_slices["plane"].map(
        {plane: i for i, plane in enumerate(PLANE_ORDER)}
    )
    visit_slices = visit_slices.sort_values(["plane_order", "slice_idx"]).reset_index(drop=True)
    
    slice_meta = []
    for _, slice_row in visit_slices.iterrows():
        slice_meta.append({
            "plane": slice_row["plane"],
            "slice_idx": int(slice_row["slice_idx"]),
            "axis_index": int(slice_row["axis_index"]),
            "img_path": slice_row["img_path"],
            "x_center": int(slice_row["x_center"]),
            "y_center": int(slice_row["y_center"]),
            "z_center": int(slice_row["z_center"]),
        })
    
    visit_record = {
        "scan_uid": scan_uid,
        "label": int(visit_row["diagnosis_visit"]),
        "dataset": visit_row["dataset"],
        "site": visit_row["site"],
        "subject_id": visit_row["subject_id"],
        "visit_code": visit_row["visit_code"],
        "split": visit_row["visit_split"],
        "cv_fold": int(visit_row["visit_cv_fold"]),
        "slice_meta": slice_meta,
    }
    
    visit_records.append(visit_record)
    n_valid += 1

print(f"\n✓ Visit records built:")
print(f"  Valid: {n_valid}")
print(f"  Invalid (skipped): {n_invalid}")

# ============================================================================
# STEP 3: SPLIT RECORDS BY SPLIT AND FOLD
# ============================================================================

print("\n[STEP 3] Organizing records by split and fold...")

trainval_records = [r for r in visit_records if r["split"] == "trainval"]
test_records = [r for r in visit_records if r["split"] == "test"]

print(f"  Train+Val records: {len(trainval_records)}")
print(f"  Test records: {len(test_records)}")

fold_records = defaultdict(lambda: {"train": [], "val": []})

for record in trainval_records:
    fold = record["cv_fold"]
    fold_records[fold]["val"].append(record)
    
    for other_fold in range(5):
        if other_fold != fold:
            fold_records[other_fold]["train"].append(record)

print(f"\n--- Per-Fold Distribution ---")
for fold in range(5):
    print(f"  Fold {fold}: Train={len(fold_records[fold]['train'])}, Val={len(fold_records[fold]['val'])}")

# ============================================================================
# STEP 4: ENHANCED DATASET CLASS
# ============================================================================

class TriPlanarVisitDataset(Dataset):
    """Enhanced dataset with visit-level augmentation."""
    
    def __init__(self, visit_records, transform=None, mode="train"):
        self.visit_records = visit_records
        self.transform = transform
        self.mode = mode
        
        for rec in self.visit_records:
            assert len(rec["slice_meta"]) == TOTAL_SLICES
    
    def __len__(self):
        return len(self.visit_records)
    
    def __getitem__(self, idx):
        rec = self.visit_records[idx]
        
        # Load all 32 slices as PIL Images
        images_pil = []
        for slice_info in rec["slice_meta"]:
            img_path = slice_info["img_path"]
            
            try:
                img = Image.open(img_path).convert("L")
            except:
                img = Image.new("L", TARGET_SIZE, color=0)
            
            images_pil.append(img)
        
        # Apply visit-level transform (handles augmentation internally)
        if self.transform is not None:
            stacked_tensor = self.transform(images_pil)  # Returns [32, H, W]
        else:
            # No transform: just convert to tensor
            tensor_slices = []
            for img in images_pil:
                img_array = np.array(img, dtype=np.float32) / 255.0
                tensor_slices.append(torch.from_numpy(img_array))
            stacked_tensor = torch.stack(tensor_slices, dim=0)
        
        label = rec["label"]
        label_tensor = torch.tensor(label, dtype=torch.long)
        
        meta = {
            "scan_uid": rec["scan_uid"],
            "subject_id": rec["subject_id"],
            "visit_code": rec["visit_code"],
            "dataset": rec["dataset"],
            "site": rec["site"],
            "split": rec["split"],
            "cv_fold": rec["cv_fold"],
            "slice_meta": rec["slice_meta"],
        }
        
        return stacked_tensor, label_tensor, meta

# ============================================================================
# STEP 5: TRANSFORMS
# ============================================================================

print("\n[STEP 4] Defining transforms...")

train_transform = VisitTransform(is_train=True)
val_transform = VisitTransform(is_train=False)
test_transform = VisitTransform(is_train=False)

print(f"  ✓ Train transform: Geometric + Intensity augmentation")
print(f"    - Geometric: HFlip, Rotation ±10°, Affine ±5%")
print(f"    - Intensity: Brightness/Contrast ±10%, Gaussian noise σ=0.02")
print(f"  ✓ Val/Test transform: No augmentation")

# ============================================================================
# STEP 6: COMPUTE CLASS WEIGHTS PER FOLD
# ============================================================================

print("\n[STEP 5] Computing class weights per fold...")

fold_class_weights = {}

for fold in range(5):
    train_recs = fold_records[fold]["train"]
    labels = [r["label"] for r in train_recs]
    n_total = len(labels)
    n_cn = labels.count(0)
    n_ad = labels.count(1)
    
    w_cn = n_total / (2 * n_cn) if n_cn > 0 else 1.0
    w_ad = n_total / (2 * n_ad) if n_ad > 0 else 1.0
    
    fold_class_weights[fold] = {
        "CN": w_cn,
        "AD": w_ad,
        "tensor": torch.tensor([w_cn, w_ad], dtype=torch.float32),
    }
    
    print(f"  Fold {fold}:")
    print(f"    Train visits: CN={n_cn}, AD={n_ad}, Total={n_total}")
    print(f"    Class weights: CN={w_cn:.4f}, AD={w_ad:.4f}")

# ============================================================================
# STEP 7: BUILD DATASETS AND DATALOADERS
# ============================================================================

print("\n[STEP 6] Building datasets and dataloaders per fold...")

fold_loaders = {}

for fold in range(5):
    print(f"\n  --- Fold {fold} ---")
    
    train_recs = fold_records[fold]["train"]
    val_recs = fold_records[fold]["val"]
    
    train_dataset = TriPlanarVisitDataset(train_recs, transform=train_transform, mode="train")
    val_dataset = TriPlanarVisitDataset(val_recs, transform=val_transform, mode="val")
    
    print(f"    Train dataset: {len(train_dataset)} visits")
    print(f"    Val dataset: {len(val_dataset)} visits")
    
    train_labels = [r["label"] for r in train_recs]
    n_cn_train = train_labels.count(0)
    n_ad_train = train_labels.count(1)
    
    sample_weights = [
        1.0 / n_ad_train if label == 1 else 1.0 / n_cn_train
        for label in train_labels
    ]
    
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(train_labels), replacement=True)
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        sampler=sampler,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        collate_fn=custom_collate_fn
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        collate_fn=custom_collate_fn
    )
    
    fold_loaders[fold] = {
        "train_dataset": train_dataset,
        "val_dataset": val_dataset,
        "train_loader": train_loader,
        "val_loader": val_loader,
        "class_weights": fold_class_weights[fold],
    }
    
    print(f"    ✓ Train loader: {len(train_loader)} batches")
    print(f"    ✓ Val loader: {len(val_loader)} batches")

# ============================================================================
# STEP 8: BUILD TEST DATASET AND LOADER
# ============================================================================

print("\n[STEP 7] Building test dataset and loader...")

test_dataset = TriPlanarVisitDataset(test_records, transform=test_transform, mode="test")

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=custom_collate_fn
)

print(f"  ✓ Test dataset: {len(test_dataset)} visits")
print(f"  ✓ Test loader: {len(test_loader)} batches")

test_labels = [r["label"] for r in test_records]
n_cn_test = test_labels.count(0)
n_ad_test = test_labels.count(1)
print(f"  Test distribution: CN={n_cn_test}, AD={n_ad_test}")

# ============================================================================
# STEP 9: QC VALIDATION
# ============================================================================

print("\n" + "=" * 80)
print("QUALITY CONTROL & VALIDATION")
print("=" * 80)

# QC 1: Subject leakage
print("\n--- QC 1: Subject-Level Leakage Check ---")
all_folds_ok = True
for fold in range(5):
    train_subjects = set([r["subject_id"] for r in fold_records[fold]["train"]])
    val_subjects = set([r["subject_id"] for r in fold_records[fold]["val"]])
    test_subjects = set([r["subject_id"] for r in test_records])
    
    if (train_subjects & val_subjects) or (train_subjects & test_subjects) or (val_subjects & test_subjects):
        all_folds_ok = False

if all_folds_ok:
    print(f"  ✓ No subject leakage detected across any fold")

# QC 2: Batch shape
print("\n--- QC 2: Batch Shape Validation ---")
fold_0_train_loader = fold_loaders[0]["train_loader"]
sample_batch = next(iter(fold_0_train_loader))
images, labels, metas = sample_batch

print(f"  Sample batch from Fold 0 train:")
print(f"    Images shape: {images.shape}")
print(f"    Images range: [{images.min():.3f}, {images.max():.3f}]")

if images.shape[1:] == (TOTAL_SLICES, TARGET_SIZE[0], TARGET_SIZE[1]):
    print(f"  ✓ Image shape matches expected")

# QC 3: Augmentation verification
print("\n--- QC 3: Augmentation Verification ---")
print(f"  Testing augmentation consistency across slices...")

# Get two batches and check if augmentation varies
batch1 = next(iter(fold_0_train_loader))
batch2 = next(iter(fold_0_train_loader))

images1, _, _ = batch1
images2, _, _ = batch2

# Check if batches are different (augmentation is working)
diff = torch.abs(images1[0] - images2[0]).mean()
print(f"  Mean difference between two samples: {diff:.4f}")

if diff > 0.01:
    print(f"  ✓ Augmentation is producing varied samples")
else:
    print(f"  ⚠️  Samples may be too similar")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("SNIPPET S5: COMPLETE ✓ (ENHANCED AUGMENTATION)")
print("=" * 80)

print(f"\n📊 Summary:")
print(f"  • Total visits: {len(visit_records)}")
print(f"  • Enhanced augmentation:")
print(f"    - Geometric: HFlip, Rotation, Affine (consistent across slices)")
print(f"    - Intensity: Brightness/Contrast jitter, Gaussian noise")
print(f"  • Class weights + WeightedRandomSampler enabled")
print(f"\n  Available objects:")
print(f"    - fold_loaders[0-4]: train/val loaders with enhanced augmentation")
print(f"    - test_loader: locked test set")
print(f"    - fold_class_weights[0-4]: for loss functions")
print(f"\n  Ready for S6 (Model + Training)")
print("=" * 80)

SNIPPET S5: TRI-PLANAR VISIT DATASET & DATALOADERS (ENHANCED)

Configuration:
  Batch size: 8
  Num workers: 4
  Target size: (160, 160)
  Slices per visit: 32
  Plane order: ['axial', 'coronal', 'sagittal']

[STEP 1] Building visit-level index...
  Loaded visits_with_splits.csv: 575 rows
  Visits usable for modeling: 517
  Loaded slices_with_splits.csv: 16544 rows

[STEP 2] Building visit records with canonical slice ordering...

✓ Visit records built:
  Valid: 517
  Invalid (skipped): 0

[STEP 3] Organizing records by split and fold...
  Train+Val records: 416
  Test records: 101

--- Per-Fold Distribution ---
  Fold 0: Train=330, Val=86
  Fold 1: Train=333, Val=83
  Fold 2: Train=344, Val=72
  Fold 3: Train=331, Val=85
  Fold 4: Train=326, Val=90

[STEP 4] Defining transforms...
  ✓ Train transform: Geometric + Intensity augmentation
    - Geometric: HFlip, Rotation ±10°, Affine ±5%
    - Intensity: Brightness/Contrast ±10%, Gaussian noise σ=0.02
  ✓ Val/Test transform: No augmentat

In [7]:
# ============================================================================
# SNIPPET S6: ATTENTION-BASED TRI-PLANAR NET (FIXED - NO COLLAPSE)
# ============================================================================

import os
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, confusion_matrix
from tqdm import tqdm

print("=" * 80)
print("SNIPPET S6: ATTENTION-BASED TRI-PLANAR NET (FIXED)")
print("=" * 80)

# ============================================================================
# CONFIGURATION
# ============================================================================

NUM_EPOCHS = 30
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BACKBONES = ["efficientnet_b0", "vgg16_bn", "densenet121", "resnet18"]
ATTENTION_DIM = 256
DROPOUT_RATE = 0.3

# KEY FIXES
USE_CLASS_WEIGHTS = False  # ← FIX 1: Let sampler handle imbalance
FREEZE_BACKBONE_EPOCHS = NUM_EPOCHS  # ← FIX 2: Keep backbone frozen
USE_FOCAL_LOSS = False

CHECKPOINT_DIR = "checkpoints_s6_fixed"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print(f"\nConfiguration:")
print(f"  Device: {DEVICE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Backbones: {BACKBONES}")
print(f"  USE_CLASS_WEIGHTS: {USE_CLASS_WEIGHTS} ← Sampler handles imbalance")
print(f"  FREEZE_BACKBONE_EPOCHS: {FREEZE_BACKBONE_EPOCHS} ← Keep frozen")

# ============================================================================
# BACKBONE FACTORY
# ============================================================================

def build_backbone(backbone_name):
    if backbone_name == "efficientnet_b0":
        from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
        backbone = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
        feature_extractor = nn.Sequential(
            backbone.features,
            nn.AdaptiveAvgPool2d(1)
        )
        feature_dim = 1280
    
    elif backbone_name == "vgg16_bn":
        from torchvision.models import vgg16_bn, VGG16_BN_Weights
        backbone = vgg16_bn(weights=VGG16_BN_Weights.IMAGENET1K_V1)
        feature_extractor = nn.Sequential(
            backbone.features,
            nn.AdaptiveAvgPool2d(1)
        )
        feature_dim = 512
    
    elif backbone_name == "densenet121":
        from torchvision.models import densenet121, DenseNet121_Weights
        backbone = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
        feature_extractor = nn.Sequential(
            backbone.features,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1)
        )
        feature_dim = 1024
    
    elif backbone_name == "resnet18":
        from torchvision.models import resnet18, ResNet18_Weights
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        feature_extractor = nn.Sequential(*list(backbone.children())[:-1])
        feature_dim = 512
    
    else:
        raise ValueError(f"Unsupported backbone: {backbone_name}")
    
    return feature_extractor, feature_dim

# ============================================================================
# MODEL
# ============================================================================

class TriPlanarAttentionNet(nn.Module):
    def __init__(self, backbone_name="efficientnet_b0", attn_dim=256, dropout=0.3):
        super(TriPlanarAttentionNet, self).__init__()
        
        self.backbone_name = backbone_name
        self.backbone, self.feature_dim = build_backbone(backbone_name)
        
        self.attention = nn.Sequential(
            nn.Linear(self.feature_dim, attn_dim),
            nn.Tanh(),
            nn.Linear(attn_dim, 1)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(self.feature_dim, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 2)
        )
    
    def encode_slices(self, x):
        B, K, H, W = x.shape
        x = x.view(B * K, 1, H, W)
        x = x.repeat(1, 3, 1, 1)
        z = self.backbone(x)
        z = z.view(B * K, self.feature_dim)
        feats = z.view(B, K, self.feature_dim)
        return feats
    
    def forward(self, x):
        feats = self.encode_slices(x)
        attn_scores = self.attention(feats)
        attn_weights = F.softmax(attn_scores, dim=1).squeeze(-1)
        weighted_feats = attn_weights.unsqueeze(-1) * feats
        v = torch.sum(weighted_feats, dim=1)
        logits = self.classifier(v)
        return logits, attn_weights

# ============================================================================
# METRICS
# ============================================================================

def compute_metrics(labels, preds, probs=None):
    labels = np.array(labels)
    preds = np.array(preds)
    
    acc = accuracy_score(labels, preds)
    bal_acc = balanced_accuracy_score(labels, preds)
    
    tn, fp, fn, tp = confusion_matrix(labels, preds, labels=[0, 1]).ravel()
    
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    
    if probs is not None:
        try:
            auc = roc_auc_score(labels, probs)
        except:
            auc = np.nan
    else:
        auc = np.nan
    
    return {
        "accuracy": acc,
        "balanced_accuracy": bal_acc,
        "sensitivity": sensitivity,
        "specificity": specificity,
        "auc": auc,
    }

# ============================================================================
# TRAINING FUNCTION
# ============================================================================

def train_fold(fold, train_loader, val_loader, class_weights, 
               backbone_name="efficientnet_b0", num_epochs=NUM_EPOCHS):
    
    print(f"\n{'=' * 80}")
    print(f"TRAINING FOLD {fold} - {backbone_name.upper()}")
    print(f"{'=' * 80}")
    
    # Initialize model
    model = TriPlanarAttentionNet(
        backbone_name=backbone_name,
        attn_dim=ATTENTION_DIM,
        dropout=DROPOUT_RATE
    )
    model = model.to(DEVICE)
    
    # Freeze backbone
    if FREEZE_BACKBONE_EPOCHS > 0:
        print(f"  Freezing backbone (will remain frozen all {NUM_EPOCHS} epochs)")
        for param in model.backbone.parameters():
            param.requires_grad = False
    
    # Optimizer
    optimizer = Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )
    
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)
    
    # Loss function (FIX: no class weights)
    if USE_CLASS_WEIGHTS:
        criterion = nn.CrossEntropyLoss(weight=class_weights.to(DEVICE))
        print(f"  Using Weighted CE: w_CN={class_weights[0]:.4f}, w_AD={class_weights[1]:.4f}")
    else:
        criterion = nn.CrossEntropyLoss()
        print(f"  Using UNWEIGHTED CrossEntropyLoss (sampler handles imbalance)")
    
    # Tracking
    best_val_bal_acc = -np.inf
    best_state = None
    history = []
    
    # Training loop
    for epoch in range(1, num_epochs + 1):
        print(f"\n--- Epoch {epoch}/{num_epochs} ---")
        
        # ===== TRAINING PHASE =====
        model.train()
        
        train_loss_accum = 0.0
        train_labels_all = []
        train_preds_all = []
        train_n_samples = 0
        
        for images, labels, metas in tqdm(train_loader, desc=f"Train", leave=False):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            
            logits, attn_weights = model(images)
            loss = criterion(logits, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            batch_size = labels.size(0)
            train_loss_accum += loss.item() * batch_size
            train_n_samples += batch_size
            
            train_labels_all.extend(labels.cpu().numpy())
            train_preds_all.extend(torch.argmax(logits, dim=1).cpu().numpy())
        
        train_loss = train_loss_accum / train_n_samples
        train_metrics = compute_metrics(train_labels_all, train_preds_all)
        
        # ===== VALIDATION PHASE =====
        model.eval()
        
        val_loss_accum = 0.0
        val_labels_all = []
        val_logits_all = []
        val_n_samples = 0
        
        with torch.no_grad():
            for images, labels, metas in tqdm(val_loader, desc=f"Val", leave=False):
                images = images.to(DEVICE)
                labels = labels.to(DEVICE)
                
                logits, attn_weights = model(images)
                loss = criterion(logits, labels)
                
                batch_size = labels.size(0)
                val_loss_accum += loss.item() * batch_size
                val_n_samples += batch_size
                
                val_labels_all.extend(labels.cpu().numpy())
                val_logits_all.append(logits.cpu())
        
        val_loss = val_loss_accum / val_n_samples
        
        # Compute predictions
        val_logits_all = torch.cat(val_logits_all, dim=0)
        val_probs = F.softmax(val_logits_all, dim=1)[:, 1].numpy()
        val_preds = torch.argmax(val_logits_all, dim=1).numpy()
        
        val_metrics = compute_metrics(val_labels_all, val_preds, val_probs)
        
        # DEBUG: Check prediction distribution
        unique_preds, pred_counts = np.unique(val_preds, return_counts=True)
        pred_dist = dict(zip(unique_preds, pred_counts))
        
        # Update scheduler
        scheduler.step(val_metrics["balanced_accuracy"])
        
        # Track best model
        if val_metrics["balanced_accuracy"] > best_val_bal_acc:
            best_val_bal_acc = val_metrics["balanced_accuracy"]
            best_state = deepcopy(model.state_dict())
            print(f"  ✓ New best Val Bal Acc: {best_val_bal_acc:.4f}")
        
        # Log metrics
        epoch_metrics = {
            "epoch": epoch,
            "train_loss": train_loss,
            "train_acc": train_metrics["accuracy"],
            "train_bal_acc": train_metrics["balanced_accuracy"],
            "val_loss": val_loss,
            "val_acc": val_metrics["accuracy"],
            "val_bal_acc": val_metrics["balanced_accuracy"],
            "val_auc": val_metrics["auc"],
            "val_sensitivity": val_metrics["sensitivity"],
            "val_specificity": val_metrics["specificity"],
        }
        history.append(epoch_metrics)
        
        # Print summary
        print(f"  Train: Loss={train_loss:.4f}, Acc={train_metrics['accuracy']:.4f}, BalAcc={train_metrics['balanced_accuracy']:.4f}")
        print(f"  Val:   Loss={val_loss:.4f}, Acc={val_metrics['accuracy']:.4f}, BalAcc={val_metrics['balanced_accuracy']:.4f}, AUC={val_metrics['auc']:.4f}")
        print(f"         Sens={val_metrics['sensitivity']:.4f}, Spec={val_metrics['specificity']:.4f}")
        print(f"  Val Pred Distribution: CN={pred_dist.get(0, 0)}, AD={pred_dist.get(1, 0)} (total={val_n_samples})")
    
    print(f"\n✓ Fold {fold} complete. Best Val Bal Acc: {best_val_bal_acc:.4f}")
    
    return best_state, history

# ============================================================================
# MAIN TRAINING LOOP
# ============================================================================

print("\n" + "=" * 80)
print(f"TRAINING {len(BACKBONES)} BACKBONES × 5 FOLDS")
print("=" * 80)

all_results = {}

for backbone_name in BACKBONES:
    print(f"\n{'#' * 80}")
    print(f"BACKBONE: {backbone_name.upper()}")
    print(f"{'#' * 80}")
    
    best_model_states = {}
    all_histories = {}
    
    for fold in range(5):
        train_loader = fold_loaders[fold]["train_loader"]
        val_loader = fold_loaders[fold]["val_loader"]
        class_weights = fold_class_weights[fold]["tensor"]
        
        best_state, history = train_fold(
            fold=fold,
            train_loader=train_loader,
            val_loader=val_loader,
            class_weights=class_weights,
            backbone_name=backbone_name,
            num_epochs=NUM_EPOCHS
        )
        
        checkpoint_path = os.path.join(CHECKPOINT_DIR, f"{backbone_name}_fold_{fold}_best.pth")
        torch.save({
            "backbone": backbone_name,
            "fold": fold,
            "model_state_dict": best_state,
            "history": history,
            "best_val_bal_acc": max([h["val_bal_acc"] for h in history]),
        }, checkpoint_path)
        
        best_model_states[fold] = best_state
        all_histories[fold] = history
    
    all_results[backbone_name] = {
        "model_states": best_model_states,
        "histories": all_histories
    }
    
    # CV Summary
    cv_results = []
    for fold in range(5):
        history = all_histories[fold]
        best_epoch_metrics = max(history, key=lambda x: x["val_bal_acc"])
        cv_results.append({
            "backbone": backbone_name,
            "fold": fold,
            "val_bal_acc": best_epoch_metrics["val_bal_acc"],
            "val_auc": best_epoch_metrics["val_auc"],
            "val_sensitivity": best_epoch_metrics["val_sensitivity"],
            "val_specificity": best_epoch_metrics["val_specificity"],
        })
    
    cv_df = pd.DataFrame(cv_results)
    print(f"\n{backbone_name.upper()} Summary:")
    print(f"  Bal Acc: {cv_df['val_bal_acc'].mean():.4f} ± {cv_df['val_bal_acc'].std():.4f}")
    print(f"  AUC: {cv_df['val_auc'].mean():.4f} ± {cv_df['val_auc'].std():.4f}")
    
    cv_df.to_csv(f"cv_results_{backbone_name}_fixed.csv", index=False)

print("\n" + "=" * 80)
print("SNIPPET S6: COMPLETE ✓")
print("=" * 80)


SNIPPET S6: ATTENTION-BASED TRI-PLANAR NET (FIXED)

Configuration:
  Device: cuda
  Epochs: 30
  Learning rate: 0.0001
  Backbones: ['efficientnet_b0', 'vgg16_bn', 'densenet121', 'resnet18']
  USE_CLASS_WEIGHTS: False ← Sampler handles imbalance
  FREEZE_BACKBONE_EPOCHS: 30 ← Keep frozen

TRAINING 4 BACKBONES × 5 FOLDS

################################################################################
BACKBONE: EFFICIENTNET_B0
################################################################################

TRAINING FOLD 0 - EFFICIENTNET_B0


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 163MB/s]


  Freezing backbone (will remain frozen all 30 epochs)
  Using UNWEIGHTED CrossEntropyLoss (sampler handles imbalance)

--- Epoch 1/30 ---


  ✓ New best Val Bal Acc: 0.5000
  Train: Loss=0.6682, Acc=0.6121, BalAcc=0.5994
  Val:   Loss=92.5684, Acc=0.8721, BalAcc=0.5000, AUC=0.5000
         Sens=0.0000, Spec=1.0000
  Val Pred Distribution: CN=86, AD=0 (total=86)

--- Epoch 2/30 ---


  ✓ New best Val Bal Acc: 0.5533
  Train: Loss=0.6273, Acc=0.6606, BalAcc=0.6609
  Val:   Loss=209.7515, Acc=0.2209, BalAcc=0.5533, AUC=0.5533
         Sens=1.0000, Spec=0.1067
  Val Pred Distribution: CN=8, AD=78 (total=86)

--- Epoch 3/30 ---


...

  Train: Loss=0.3865, Acc=0.8374, BalAcc=0.8378
  Val:   Loss=0.4034, Acc=0.8222, BalAcc=0.7363, AUC=0.8492
         Sens=0.6154, Spec=0.8571
  Val Pred Distribution: CN=71, AD=19 (total=90)

--- Epoch 27/30 ---


  Train: Loss=0.3784, Acc=0.8282, BalAcc=0.8233
  Val:   Loss=0.3452, Acc=0.8667, BalAcc=0.7303, AUC=0.8442
         Sens=0.5385, Spec=0.9221
  Val Pred Distribution: CN=77, AD=13 (total=90)

--- Epoch 28/30 ---


  Train: Loss=0.3410, Acc=0.8650, BalAcc=0.8620
  Val:   Loss=0.3584, Acc=0.8667, BalAcc=0.7303, AUC=0.8402
         Sens=0.5385, Spec=0.9221
  Val Pred Distribution: CN=77, AD=13 (total=90)

--- Epoch 29/30 ---


  Train: Loss=0.3554, Acc=0.8466, BalAcc=0.8478
  Val:   Loss=0.3904, Acc=0.8222, BalAcc=0.7363, AUC=0.8531
         Sens=0.6154, Spec=0.8571
  Val Pred Distribution: CN=71, AD=19 (total=90)

--- Epoch 30/30 ---


  Train: Loss=0.3942, Acc=0.8221, BalAcc=0.8210
  Val:   Loss=0.4049, Acc=0.8444, BalAcc=0.7493, AUC=0.8442
         Sens=0.6154, Spec=0.8831
  Val Pred Distribution: CN=73, AD=17 (total=90)

✓ Fold 4 complete. Best Val Bal Acc: 0.7937

RESNET18 Summary:
  Bal Acc: 0.6810 ± 0.0748
  AUC: 0.7197 ± 0.1042

SNIPPET S6: COMPLETE ✓
